# INFORMAZIONI SUL SET DI DATI

Dichiarazione problema:
Una grande azienda denominata XYZ impiega, in un dato momento, circa 4000 dipendenti.
Tuttavia, ogni anno, circa il 15% dei suoi dipendenti lascia l'azienda e deve essere sostituito con il pool di talenti
disponibile nel mercato del lavoro. Il management ritiene che questo livello di logoramento
(dipendenti che se ne vanno, da soli o perché sono stati licenziati) sia negativo per l'azienda, per i seguenti motivi:

 1 - I progetti degli ex dipendenti vengono ritardati, il che rende difficile rispettare le scadenze, con conseguente perdita di reputazione tra consumatori e partner;

 2 - Occorre mantenere un reparto consistente, ai fini del reclutamento di nuovi talenti;

 3 - Il più delle volte, i nuovi dipendenti devono essere formati per il lavoro e/o avere il tempo di ambientarsi in azienda.

Pertanto, la direzione ha incaricato una società di analisi delle risorse umane
per capire su quali fattori dovrebbero concentrarsi, al fine di frenare l'attrito.
In altre parole, vogliono sapere quali cambiamenti dovrebbero apportare al loro posto di lavoro, al fine di convincere
la maggior parte dei loro dipendenti a restare.
Inoltre, vogliono sapere quale di queste variabili è più importante e deve essere affrontata immediatamente.

Obiettivo del caso di studio

È necessario modellare la probabilità di logoramento utilizzando una regressione logistica.
I risultati così ottenuti serviranno al management per capire quali cambiamenti dovrebbero apportare
al proprio posto di lavoro, al fine di far rimanere la maggior parte dei propri dipendenti.

# Importazione e unione dei dati

In [1]:
# Soppressione Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import librerie Pandas, NumPy, Seaborn e Matplot
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, datetime as datetime, re as re, scipy.stats as stats

In [3]:
import itertools
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# Lettura tabelle
ingresso = pd.read_csv('in_time.csv') #orario di ingresso dei dipendenti
uscita = pd.read_csv('out_time.csv') #orario di uscita dei dipendenti
sondaggio_manager = pd.read_csv('manager_survey_data.csv') #valutazione dei dipendenti sui loro manager
sondaggio_dipendenti = pd.read_csv('employee_survey_data.csv') #soddisfazione dei dipendenti sull'ambiente, il lavoro
# e l'equilibrio vita-lavoro
dati_generali = pd.read_csv('general_data.csv') #dati generali riguardo i dipendenti
data_dictionary = pd.read_excel('data_dictionary.xlsx') #dizionario interpretazioni variabili

In [6]:
#si visualizza le dimensioni e le info del DataFrame attraverso l'attributo .shape e .info che restituisce il numero di righe e colonne.
ingresso.shape
ingresso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 262 entries, Unnamed: 0 to 2015-12-31
dtypes: float64(12), int64(1), object(249)
memory usage: 8.8+ MB


In [7]:
# Si sostituiscono i valori nulli con una data fittizia e si convertono in formato data
ingresso = ingresso.fillna(datetime.datetime(1970,1,1))
ingresso.iloc[:, 1:] = ingresso.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

In [8]:
# Si esegue la stessa pulizia per il DF uscita
uscita = uscita.fillna(datetime.datetime(1970,1,1))
uscita.iloc[:, 1:] = uscita.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

In [ ]:
#Si uniscono i due DF ingresso e uscita: pd.concat() viene utilizzato per aggiungere righe di altri dataframe
#alla fine del primo dataframe specificato, restituendo un nuovo dataframe. pd.concat ha sostituito df.append.
ingresso_uscita = pd.concat([ingresso,uscita])

In [ ]:
# conversione type in datetime
ingresso_uscita = ingresso_uscita.convert_dtypes(datetime)

In [ ]:
# Si rimuovono le colonne con solo valori fittizi
ingresso_uscita.drop(['2015-01-01', '2015-01-14','2015-01-26','2015-03-05',
             '2015-05-01','2015-07-17','2015-09-17','2015-10-02',
              '2015-11-09','2015-11-10','2015-11-11','2015-12-25'
             ], axis = 1,inplace=True)

In [ ]:
# Si calcola la differenza di un elemento DataFrame rispetto a un altro elemento in DataFrame
# (l'impostazione predefinita è l'elemento nella riga precedente).
# "Periods" serve per indicare il numero di periodi (in questo caso corrispondente alle righe)
# da spostare per il calcolo della differenza.
ingresso_uscita = ingresso_uscita.diff(periods=4410)
# L' indicizzatore iloc per Pandas Dataframe viene utilizzato per  l'indicizzazione/selezione
# in base alla posizione basata su numeri interi.
# La sintassi dell'indicizzatore iloc è data.iloc[<selezione riga>, <selezione colonna>]
ingresso_uscita=ingresso_uscita.iloc[4410:]
# Pandas reset_index()è un metodo per reimpostare l'indice di un data frame.
# Il metodo reset_index() imposta un elenco di numeri interi che vanno da 0 alla lunghezza dei dati come indice.
ingresso_uscita.reset_index(inplace=True)
ingresso_uscita.head()

In [ ]:
# rimozione colonne index e Unnamed
ingresso_uscita.drop(columns=['index','Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# Verifica struttura nuovo DF
ingresso_uscita.shape

In [ ]:
ingresso_uscita.info()

In [ ]:
# Nuova colonna con la media per ogni riga
ingresso_uscita['Media Durate'] = ingresso_uscita.mean(axis=1)

In [ ]:
# Si convertono i valori del DF in formato hh:mm:ss
ingresso_uscita = ingresso_uscita.applymap(lambda x: f"{x.components.hours:02d}:{x.components.minutes:02d}:{x.components.seconds:02d}" if pd.notnull(x) else '')

In [ ]:
ingresso_uscita['Media Durate'].head()

In [ ]:
# Nuova colonna con numero di ore per ogni valore nella colonna 'Media Durate'
ingresso_uscita['Nr Ore']=ingresso_uscita['Media Durate']/np.timedelta64(1, 'h')
ingresso_uscita.head()

In [ ]:
# Reset dell'indice
ingresso_uscita.reset_index(inplace=True)

In [ ]:
columns_to_keep = ['index', 'Nr Ore']
ingresso_uscita = ingresso_uscita[columns_to_keep]
ingresso_uscita.rename(columns={'index': 'EmployeeID'},inplace=True)
# Tabella riassuntiva in cui per ogni ID dipendente è affiancato il numero di ore medie lavorative
ingresso_uscita.head()

In [ ]:
# Si combinano i dataframe a disposizione in un unico DF
df = pd.merge(sondaggio_dipendenti, dati_generali, how='inner', on='EmployeeID')
hr = pd.merge(sondaggio_manager, df, how='inner', on='EmployeeID')
hr = pd.merge(ingresso_uscita, hr, how='inner', on='EmployeeID')
hr.head()

In [ ]:
hr.info()

In [ ]:
# Correzione del tipo di dati per le variabili
hr['JobLevel']=hr['JobLevel'].astype(str)

In [ ]:
# Si crea DataFrame con valori decodificati
hr_var_dec = hr.copy()
hr_var_dec['Education'] = hr_var_dec['Education'].replace({ 1 : 'Below College', 2: 'College',3: 'Bachelor',4: 'Master',5 : 'Doctor'})
hr_var_dec['EnvironmentSatisfaction'] = hr_var_dec['EnvironmentSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hr_var_dec['JobInvolvement'] = hr_var_dec['JobInvolvement'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hr_var_dec['JobSatisfaction'] = hr_var_dec['JobSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hr_var_dec['PerformanceRating'] = hr_var_dec['PerformanceRating'].replace({ 1 : 'Low', 2: 'Good',3: 'Excellent',4: 'Outstanding'})
hr_var_dec['WorkLifeBalance'] = hr_var_dec['WorkLifeBalance'].replace({ 1 : 'Bad', 2: 'Good',3: 'Better',4: 'Best'})

In [ ]:
hr_var_dec.head()

In [ ]:
# Conta dei dipendenti
hr['EmployeeCount'].value_counts(ascending=False)

In [ ]:
# Conta dei dipendenti maggiorenni
hr['Over18'].value_counts(ascending=False)

In [ ]:
# Conta delle ore di lavoro standard dei dipendenti
hr['StandardHours'].value_counts(ascending=False)

In [ ]:
# Rimozione delle colonne non necessarie dai due DF
hr.drop(['EmployeeID', 'EmployeeCount','StandardHours','Over18'], axis = 1,inplace=True)
hr_var_dec.drop(['EmployeeID', 'EmployeeCount','StandardHours','Over18'], axis = 1,inplace=True)

# Controllo dei DataFrame

In [ ]:
# Si controllano le dimensioni del DataFrame
hr.shape

In [ ]:
hr_var_dec.shape

In [ ]:
# Si osserva un quadro generale delle statistiche descrittive
hr.describe()

In [ ]:
# Si osserva le informazioni riguardo il DataFrame
hr.info()

In [ ]:
hr_var_dec.info()

In [ ]:
# Notando dalle info che alcune variabili hanno valori mancanti si verifica quali sono
vars_with_missing = [col for col in hr.columns if hr[col].isnull().any()]
if not vars_with_missing:
    print("Non ci sono valori mancanti.")
else:
    print("Le seguenti variabili hanno valori mancanti:", vars_with_missing)

In [ ]:
vars_with_missing = [col for col in hr_var_dec.columns if hr_var_dec[col].isnull().any()]
if not vars_with_missing:
    print("Non ci sono valori mancanti.")
else:
    print("Le seguenti variabili hanno valori mancanti:", vars_with_missing)

## Variabile "EnvironmentSatisfaction"

In [ ]:
hr['EnvironmentSatisfaction'].value_counts(ascending=False)

In [ ]:
hr_var_dec['EnvironmentSatisfaction'].value_counts(ascending=False)

In [ ]:
hr['EnvironmentSatisfaction'].mean()

In [ ]:
hr['EnvironmentSatisfaction'].median()

In [ ]:
sns.countplot(x='EnvironmentSatisfaction',data=hr_var_dec)

In [ ]:
sns.boxplot(x='EnvironmentSatisfaction',data=hr)

Dal sondaggio dipendenti si riscontra che la media e la mediana circa il livello di soddisfazione dell'ambiente di lavoro sono rispettivamente di 2.72 e 3. Essendo dati di natura ordinale la mediana è più appropriata in quanto preserva l'ordine dei dati. Dalla Dictionary è noto che il valore 3 viene categorizzato come "High".

In [ ]:
# Il numero di valori mancanti è:
hr_var_dec['EnvironmentSatisfaction'].isnull().sum()

In [ ]:

hr['EnvironmentSatisfaction'].isnull().sum()

In [ ]:
# Si sostituisce quindi con "High" i valori mancanti
hr_var_dec['EnvironmentSatisfaction'] = hr_var_dec['EnvironmentSatisfaction'].fillna('High')
hr_var_dec['EnvironmentSatisfaction'].isnull().sum()

In [ ]:
# Si sostituisce quindi con 3 i valori mancanti
hr['EnvironmentSatisfaction'] = hr['EnvironmentSatisfaction'].fillna(3)
hr['EnvironmentSatisfaction'].isnull().sum()

## Variabile "JobSatisfaction"

In [ ]:
hr['JobSatisfaction'].value_counts(ascending=False)

In [ ]:
hr_var_dec['JobSatisfaction'].value_counts(ascending=False)

In [ ]:
hr['JobSatisfaction'].mean()

In [ ]:
hr['JobSatisfaction'].median()

In [ ]:
sns.countplot(x='JobSatisfaction',data=hr_var_dec)

In [ ]:
sns.boxplot(x='JobSatisfaction',data=hr)

Dal sondaggio dipendenti si riscontra che la media e la mediana circa il livello di soddisfazione del lavoro sono rispettivamente di 2.73 e 3. Essendo dati di natura ordinale la mediana è più appropriata in quanto preserva l'ordine dei dati. Dalla Dictionary è noto che il valore 3 viene categorizzato come "High".

In [ ]:
# Il numero di valori mancanti è:
hr['JobSatisfaction'].isnull().sum()

In [ ]:
hr_var_dec['JobSatisfaction'].isnull().sum()

In [ ]:
# Si sostituisce quindi con 3 i valori mancanti
hr['JobSatisfaction'] = hr['JobSatisfaction'].fillna(3)
hr['JobSatisfaction'].isnull().sum()

In [ ]:
# Si sostituisce quindi con "High" i valori mancanti
hr_var_dec['JobSatisfaction'] = hr_var_dec['JobSatisfaction'].fillna('High')
hr_var_dec['JobSatisfaction'].isnull().sum()

## Variabile "WorkLifeBalance"

In [ ]:
hr['WorkLifeBalance'].value_counts(ascending=False)

In [ ]:
hr_var_dec['WorkLifeBalance'].value_counts(ascending=False)

In [ ]:
hr['WorkLifeBalance'].mean()

In [ ]:
hr['WorkLifeBalance'].median()

In [ ]:
sns.countplot(x='WorkLifeBalance',data=hr_var_dec)

In [ ]:
sns.boxplot(x='WorkLifeBalance',data=hr)

Dal sondaggio dipendenti si riscontra che la media e la mediana circa il livello di equilibrio tra lavoro e vita privata sono rispettivamente di 2.76 e 3. Essendo dati di natura ordinale la mediana è più appropriata in quanto preserva l'ordine dei dati. Dalla Dictionary è noto che il valore 3 viene categorizzato come "Better".

In [ ]:
# Il numero di valori mancanti è:
hr['WorkLifeBalance'].isnull().sum()

In [ ]:
hr_var_dec['WorkLifeBalance'].isnull().sum()

In [ ]:
# Si sostituisce quindi con 3 i valori mancanti
hr['WorkLifeBalance'] = hr['WorkLifeBalance'].fillna(3)
hr['WorkLifeBalance'].isnull().sum()

In [ ]:
# Si sostituisce quindi con "Better" i valori mancanti
hr_var_dec['WorkLifeBalance'] = hr_var_dec['WorkLifeBalance'].fillna('Better').astype(str)
hr_var_dec['WorkLifeBalance'].isnull().sum()

## Variabile "NumCompaniesWorked"

In [ ]:
hr_var_dec['NumCompaniesWorked'].value_counts(ascending=False)

In [ ]:
# Grafico a barre che conta il numero di aziende in cui gli impiegati hanno lavorato in precedenza.
sns.countplot(x='NumCompaniesWorked',data=hr)

In [ ]:
sns.boxplot(x='NumCompaniesWorked',data=hr)

In [ ]:
hr['NumCompaniesWorked'].mean()

In [ ]:
hr['NumCompaniesWorked'].median()

Dall'analisi svolta, la media e la mediana del numero di compagnie in cui i dipendenti hanno lavorato sono rispettivamente 2.69 e 2. Poiché dal Boxplot si nota che i dati contengono valori estremi che potrebbero influenzare significativamente la media, la mediana risulta essere una scelta migliore. Visto che la mediana non è influenzata dagli outlier, rappresenta meglio il valore centrale della distribuzione dei dati.
Si sostituisce quindi con 2 i valori mancanti.

In [ ]:
# Il numero di valori mancanti è:
hr['NumCompaniesWorked'].isnull().sum()

In [ ]:
# Si sostituisce quindi con 2 i valori mancanti
hr['NumCompaniesWorked'] = hr['NumCompaniesWorked'].fillna(2)
hr['NumCompaniesWorked'].isnull().sum()

In [ ]:
hr_var_dec['NumCompaniesWorked'] = hr_var_dec['NumCompaniesWorked'].fillna(2)
hr_var_dec['NumCompaniesWorked'].isnull().sum()

## Variabile "TotalWorkingYears"

In [ ]:
# Si conta il numero di occorrenze per ogni valore presente nella colonna 'TotalWorkingYears' del dataset hr ordinandoli in ordine decrescente.
hr['TotalWorkingYears'].value_counts(ascending=False)

In [ ]:
# Si crea un istogramma della distribuzione dei valori della variabile "TotalWorkingYears"
plt.figure(figsize=(8,8))
ax = sns.distplot(hr['TotalWorkingYears'], hist=True, kde=False,
             bins=int(180/5), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('# of Employees')
ax.set_xlabel('TotalWorkingYears')

In [ ]:
sns.boxplot(x='TotalWorkingYears',data=hr)

In [ ]:
hr['TotalWorkingYears'].mean()

In [ ]:
hr['TotalWorkingYears'].median()

Dall'analisi svolta, la media e la mediana del numero totale di anni in cui il dipendente ha lavorato finora sono rispettivamente 11.28 e 10. Poiché dal Boxplot si nota che i dati contengono valori estremi che potrebbero influenzare significativamente la media, la mediana risulta essere una scelta migliore. Visto che la mediana non è influenzata dagli outlier, rappresenta meglio il valore centrale della distribuzione dei dati.
Si sceglie quindi 10 come valore con cui sostituire i dati mancanti.

In [ ]:
# il numero dei valori mancanti è:
hr['TotalWorkingYears'].isnull().sum()

In [ ]:
# Si sostituisce con 10 i valori mancanti
hr['TotalWorkingYears'] = hr['TotalWorkingYears'].fillna(10)
hr['TotalWorkingYears'].isnull().sum()

In [ ]:
hr_var_dec['TotalWorkingYears'] = hr_var_dec['TotalWorkingYears'].fillna(10)
hr_var_dec['TotalWorkingYears'].isnull().sum()

# Analisi esplorativa dei dati

## Intro

In [ ]:
hr_var_dec.info()

In [ ]:
hr_var_dec.head()

### Attrition

In [ ]:
# Calcola le frequenze relative e relative percentuali
attrition_relatives = hr_var_dec["Attrition"].value_counts(normalize=True)
attrition_percentages = hr_var_dec["Attrition"].value_counts(normalize=True) * 100

# Crea l'istogramma
fig, ax = plt.subplots()
bars = plt.bar(attrition_percentages.index, attrition_relatives.values, color=['green', 'red'])
plt.xlabel("Attrition")
plt.title("Employee Attrition")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars, attrition_percentages):
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di attrition
conteggio_per_attrition = hr_var_dec.groupby("Attrition").size()

# Crea una tabella
tabella_conteggio_per_attrition = pd.DataFrame(conteggio_per_attrition, columns=["Conteggio Dipendenti"])

# Stampa la tabella
print(tabella_conteggio_per_attrition)

### Age

In [ ]:
# Calcola il conteggio delle età
age_counts = hr_var_dec["Age"].value_counts().sort_index()

# Crea il diagramma a barre
plt.figure(figsize=(12, 6))  # Imposta le dimensioni del grafico
plt.bar(age_counts.index, age_counts.values, color='blue', alpha=0.7)
plt.xticks(age_counts.index, rotation=0)  # Ruota le etichette sull'asse x per la leggibilità

### Gender

In [ ]:
# Calcola le frequenze relative e relative percentuali
gender_relatives = hr_var_dec["Gender"].value_counts(normalize=True)
gender_percentages = hr_var_dec["Gender"].value_counts(normalize=True) * 100

# Crea l'istogramma
fig, ax = plt.subplots()
bars = plt.bar(gender_percentages.index, gender_relatives.values, color=['blue', 'pink'])
plt.xlabel("Gender")
plt.title("Employee Gender")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars, gender_percentages):
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di gender
conteggio_per_gender = hr_var_dec.groupby("Gender").size()

# Crea una tabella
tabella_conteggio_per_gender = pd.DataFrame(conteggio_per_gender, columns=["Conteggio Dipendenti"])

# Stampa la tabella
print(tabella_conteggio_per_gender)

### Education

In [ ]:
# Definisci l'ordine desiderato delle categorie di istruzione
order = ['Below College', 'College', 'Bachelor', 'Master', 'Doctor']

# Calcola le frequenze relative e relative percentuali
education_relatives = hr_var_dec["Education"].value_counts(normalize=True).loc[order]
education_percentages = (hr_var_dec["Education"].value_counts(normalize=True) * 100).loc[order]

# Colori personalizzati
colors = ['red', 'orange', 'yellow', (144/255, 238/255, 144/255), 'green']

# Crea l'istogramma
fig, ax = plt.subplots()
bars = sns.barplot(x=education_relatives.index, y=education_relatives.values, order=order, palette=colors)
plt.xlabel("Education")
plt.title("Employee Education")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars.patches, education_percentages):
    height = bar.get_height()
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di gender
conteggio_per_education = hr_var_dec.groupby("Education").size()

# Crea una tabella
tabella_conteggio_per_education = pd.DataFrame(conteggio_per_education, columns=["Conteggio Dipendenti per educazione"])

# Stampa la tabella
print(tabella_conteggio_per_education)

### EnvironmentSatisfaction

In [ ]:
# Definisci l'ordine desiderato delle categorie di soddisfazione dell'ambiente lavorativo
order = ['Low', 'Medium', 'High', 'Very High']

# Calcola le frequenze relative e relative percentuali
environment_relatives = hr_var_dec["EnvironmentSatisfaction"].value_counts(normalize=True).loc[order]
environment_percentages = (hr_var_dec["EnvironmentSatisfaction"].value_counts(normalize=True) * 100).loc[order]

# Colori personalizzati
colors = ['red', 'orange', 'yellow', 'green']

# Crea l'istogramma
fig, ax = plt.subplots()
bars = sns.barplot(x=environment_relatives.index, y=environment_relatives.values, order=order, palette=colors)
plt.xlabel("Satisfaction")
plt.title("Environment Satisfaction")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars.patches, environment_percentages):
    height = bar.get_height()
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di environment satisfaction
conteggio_per_EnvironmentSatisfaction = hr_var_dec.groupby("EnvironmentSatisfaction").size()

# Crea una tabella
tabella_conteggio_per_EnvironmentSatisfaction = pd.DataFrame(conteggio_per_EnvironmentSatisfaction, columns=["Conteggio Dipendenti per soddisfazione dell'ambiente"])

# Stampa la tabella
print(tabella_conteggio_per_EnvironmentSatisfaction)

### JobSatisfaction

In [ ]:
# Definisci l'ordine desiderato delle categorie di soddisfazione del lavoro
order = ['Low', 'Medium', 'High', 'Very High']

# Calcola le frequenze relative e relative percentuali
job_relatives = hr_var_dec["JobSatisfaction"].value_counts(normalize=True).loc[order]
job_percentages = (hr_var_dec["JobSatisfaction"].value_counts(normalize=True) * 100).loc[order]

# Colori personalizzati
colors = ['red', 'orange', 'yellow', 'green']

# Crea l'istogramma
fig, ax = plt.subplots()
bars = sns.barplot(x=job_relatives.index, y=job_relatives.values, order=order, palette=colors)
plt.xlabel("Satisfaction")
plt.title("Job Satisfaction")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars.patches, job_percentages):
    height = bar.get_height()
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di environment satisfaction
conteggio_per_JobSatisfaction = hr_var_dec.groupby("JobSatisfaction").size()

# Crea una tabella
tabella_conteggio_per_JobSatisfaction = pd.DataFrame(conteggio_per_JobSatisfaction, columns=["Conteggio Dipendenti per soddisfazione del lavoro"])

# Stampa la tabella
print(tabella_conteggio_per_JobSatisfaction)

### JobInvolvement

In [ ]:
# Definisci l'ordine desiderato delle categorie di coinvolgimento lavorativo
order = ['Low', 'Medium', 'High', 'Very High']

# Calcola le frequenze relative e relative percentuali
jobInv_relatives = hr_var_dec["JobInvolvement"].value_counts(normalize=True).loc[order]
jobInv_percentages = (hr_var_dec["JobInvolvement"].value_counts(normalize=True) * 100).loc[order]

# Colori personalizzati
colors = ['red', 'orange', 'yellow', 'green']

# Crea l'istogramma
fig, ax = plt.subplots()
bars = sns.barplot(x=jobInv_relatives.index, y=jobInv_relatives.values, order=order, palette=colors)
plt.xlabel("Involvement")
plt.title("Job Involvement")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars.patches, jobInv_percentages):
    height = bar.get_height()
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di Job Involvement
conteggio_per_JobInvolvement = hr_var_dec.groupby("JobInvolvement").size()

# Crea una tabella
tabella_conteggio_per_JobInvolvement = pd.DataFrame(conteggio_per_JobInvolvement, columns=["Conteggio Dipendenti per coinvolgimento lavorativo"])

# Stampa la tabella
print(tabella_conteggio_per_JobInvolvement)

### PerformanceRating

In [ ]:
hr_var_dec['PerformanceRating'].head()

In [ ]:
# Definisci l'ordine desiderato delle categorie di coinvolgimento lavorativo
order = ['Excellent', 'Outstanding']

# Calcola le frequenze relative e relative percentuali
jobInv_relatives = hr_var_dec["PerformanceRating"].value_counts(normalize=True).loc[order]
jobInv_percentages = (hr_var_dec["PerformanceRating"].value_counts(normalize=True) * 100).loc[order]

# Colori personalizzati
colors = ['yellow', 'green']

# Crea l'istogramma
fig, ax = plt.subplots()
bars = sns.barplot(x=jobInv_relatives.index, y=jobInv_relatives.values, order=order, palette=colors)
plt.xlabel("Performance")
plt.title("Performance Rating")

# Aggiungi le etichette delle percentuali sulle barre
for bar, percentage in zip(bars.patches, jobInv_percentages):
    height = bar.get_height()
    ax.annotate(f'{percentage:.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points', ha='center')

In [ ]:
# Calcola il conteggio dei dipendenti per ciascuna categoria di Job Involvement
conteggio_per_PerformanceRating = hr_var_dec.groupby("PerformanceRating").size()

# Crea una tabella
tabella_conteggio_per_PerformanceRating = pd.DataFrame(conteggio_per_PerformanceRating, columns=["Conteggio Dipendenti per valutazione delle prestazioni per l'anno scorso"])

# Stampa la tabella
print(tabella_conteggio_per_PerformanceRating)

## Matrice di correlazione dei dati quantitativi

In [ ]:
# Dataframe con soli variabili numeriche
hr_num=hr[['Attrition','Nr Ore','Age','DistanceFromHome','MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear','YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager']]

In [ ]:
# Conversione della variabile "Attrition" in formato numerico
hr_num['Attrition'] = hr_num['Attrition'].map({'No': 0, 'Yes': 1})

In [ ]:
# Mappa di calore (heatmap) della matrice di correlazione
plt.figure(figsize=(20, 8))
sns.heatmap(hr_num.corr(), annot = True, cmap="Accent", fmt='.2f')
plt.title('Matrice di Correlazione')
plt.show()

Osservando la matrice di correlazione, si possono identificare le variabili esplicative che mostrano una forte correlazione con la variabile risposta. Queste variabili possono essere candidate importanti per l'analisi successiva e possono richiedere ulteriori esplorazioni o analisi più approfondite per comprendere meglio la loro relazione con la variabile risposta.

Tuttavia, è importante notare che la correlazione non implica necessariamente una relazione di causa-effetto tra le variabili.

In [ ]:
# Lista delle variabili numeriche da testare
variabili_esplicative = ['Nr Ore','Age','DistanceFromHome','MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
                            'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear','YearsAtCompany',
                            'YearsSinceLastPromotion', 'YearsWithCurrManager']

# Lista per tenere traccia delle variabili significative
variabili_significative = []

# Calcola il test t per ciascuna variabile
for variabile in variabili_esplicative:
    attrition_yes = hr[hr['Attrition'] == 'Yes'][variabile]
    attrition_no = hr[hr['Attrition'] == 'No'][variabile]
    t_statistic, p_value = stats.ttest_ind(attrition_yes, attrition_no)

    # Verifica la significatività con un valore di soglia (ad esempio, 0.05)
    if p_value < 0.05:
        variabili_significative.append(variabile)

# Costruisci la frase risultante
risultato = "Le seguenti variabili esplicative numeriche risultano significative: " + ", ".join(variabili_significative)

# Stampa il risultato
print(risultato)


Anche dai t-test di ciascuna variabile esplicativa con la variabile risposta Attrition le variabili esplicative che risultano significative sono:

Age - MonthlyIncome - NumCompaniesWorked - PercentSalaryHike - TotalWorkingYears - TrainingTimesLastYears - YearsAtCompany - YearsSinceLastPromotion - YearsWithCurrManager

Data l'ipotesi nulla che le due distribuzioni siano uguali il p value molto piccolo suggerisce che c'è una differenza statisticamente significativa nella variabile esplicativa tra i gruppi con Attrition = 'Yes' e Attrition = 'No'. Si può dunque affermare che le due distribuzioni sono significativamente diverse.

Questo risultato suggerisce che le varibili sopra elencate potrebbero essere un fattore importante da considerare nell'analisi dell'attrition e potrebbero essere utili per comprendere meglio i motivi per cui i dipendenti lasciano l'azienda.

## Attrition & Age

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'Yes'
training_counts_yes = attrition_yes['Age'].value_counts().sort_index()

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'No'
training_counts_no = attrition_no['Age'].value_counts().sort_index()

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo grafico a barre per Attrition = 'Yes'
axes[0].bar(training_counts_yes.index, training_counts_yes.values)
axes[0].set_title('Distribuzione dell\'Età dei Dipendenti - Attrition = Yes')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')

# Crea il secondo grafico a barre per Attrition = 'No'
axes[1].bar(training_counts_no.index, training_counts_no.values)
axes[1].set_title('Distribuzione dell\'Età dei Dipendenti = No')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.tight_layout()

# Visualizza i grafici affiancati
plt.show()


In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='Age', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['Age'].mean()
stds = hr.groupby('Attrition')['Age'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, 6.5, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, 4.5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Il grafico a violino mostra la distribuzione dell'età dei dipendenti divisi in base all'attrition. La forma del violino rappresenta la densità di probabilità della distribuzione dell'età. La parte più ampia del violino indica la regione in cui si concentra la maggior parte dei dati, mentre la parte più stretta indica la regione in cui si concentra una minore quantità di dati.
Inoltre, il grafico a violino è diviso in due parti, una per i dipendenti che hanno lasciato l'azienda (Attrition = Yes) e una per quelli che non l'hanno fatto (Attrition = No). In questo modo, è possibile confrontare la distribuzione dell'età tra i due gruppi di dipendenti e vedere se ci sono differenze significative.
In questo caso la parte più ampia del violino per i dipendenti con Attrition = Yes è spostata verso il basso rispetto a quella per i dipendenti con Attrition = No, ciò potrebbe indicare che i dipendenti più giovani tendono a lasciare l'azienda più facilmente.

In [ ]:
from sklearn.cluster import KMeans

# Estrai le colonne Attrition e Age dal DataFrame hr_num
X = hr_num[['Attrition', 'Age']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 10), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

Il metodo del gomito suggerisce di classificare le età in tre classi

In [ ]:
# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=3, random_state=42)

# Addestra il modello sui dati dell'età
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi degli intervalli di età per ogni cluster
age_ranges = []
for cluster_id in range(3):
    age_values = hr.loc[hr['Cluster'] == cluster_id, 'Age']
    age_min = age_values.min()
    age_max = age_values.max()
    age_range = f"{age_min} - {age_max}"
    age_ranges.append(age_range)

print(age_ranges)

In [ ]:
# Definisci gli estremi degli intervalli di età (inclusi -> right=True)
intervalli_eta = [(18, 33), (34, 44), (45,60)]

hr['EtaIntervallo'] = pd.cut(hr['Age'], bins=[intervallo[0] for intervallo in intervalli_eta] + [intervalli_eta[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_eta], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='EtaIntervallo', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto alle classi di età, la percentuale dei dipendenti che ha lasciato lo scorso anno è  minore rispetto alla percentuale di chi è rimasto.

- Il 23% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di età 18-33.
- Il 12% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di età 45-60.
- Il 10% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di età 34-44.


In [ ]:
# Calcola la tabella di frequenza percentuali per colonna
table = pd.crosstab(hr['EtaIntervallo'], hr['Attrition'], normalize='columns') * 100

# Stampa la tabella di frequenza
print(table)

A ulteriore conferma di quanto visto dai grafici si veda la tabella a doppia entrata. Questa risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 63% ha un età compresa tra i 18 ed i 33 anni. La seconda percentuale più alta, il 22%, è di chi ha un età compresa tra i 34 ed i 44 anni.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['Age'], attrition_no['Age'])

# Stampa il risultato del test
alpha = 0.05
if p_value < alpha:
    print("La differenza nell'età tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nell'età tra i gruppi di Attrition.")

Anche il risutato che si ottiene dal t-test è in linea con quanto osservato in precedenza; sia con alpha = 0.05 che con alpha = 0.01

## Attrition & MonthlyIncome

In [ ]:
# Tasso di cambio rupie-euro
tasso_cambio = 0.011  # Esempio: 1 rupia = 0.011 euro

# Converti i valori di MonthlyIncome in euro
hr['MonthlyIncome_Euro'] = hr['MonthlyIncome'] * tasso_cambio
hr_num['MonthlyIncome_Euro'] = hr_num['MonthlyIncome'] * tasso_cambio

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo istogramma per Attrition = 'Yes'
axes[0].hist(attrition_yes['MonthlyIncome_Euro'], bins=10, edgecolor='black', alpha=0.5)
axes[0].set_title('Distribuzione del Reddito - Attrition = Yes')
axes[0].set_xlabel('MonthlyIncome_Euro')
axes[0].set_ylabel('Frequency')

# Crea il secondo istogramma per Attrition = 'No'
axes[1].hist(attrition_no['MonthlyIncome_Euro'], bins=10, edgecolor='black', alpha=0.5)
axes[1].set_title('Distribuzione del Reddito - Attrition = No')
axes[1].set_xlabel('MonthlyIncome_Euro')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.tight_layout()

# Visualizza i grafici affiancati
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='MonthlyIncome_Euro', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['MonthlyIncome_Euro'].mean()
stds = hr.groupby('Attrition')['MonthlyIncome_Euro'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -500, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -600, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

In [ ]:
# Estrai le colonne Attrition e MonthlyIncome_Euro dal DataFrame hr_num
X = hr_num[['Attrition', 'MonthlyIncome_Euro']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 10), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

Il metodo del gomito suggerisce di classificare MonthlyIncome_Euro in tre classi

In [ ]:
# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=3, random_state=42)

# Addestra il modello sui dati MonthlyIncome_Euro
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr_num['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi di MonthlyIncome_Euro per ogni cluster
MonthlyIncome_Euro_ranges = []
for cluster_id in range(3):
    MonthlyIncome_Euro_values = hr_num.loc[hr_num['Cluster'] == cluster_id, 'MonthlyIncome_Euro']
    MonthlyIncome_Euro_min = MonthlyIncome_Euro_values.min()
    MonthlyIncome_Euro_max = MonthlyIncome_Euro_values.max()
    MonthlyIncome_Euro_range = f"{MonthlyIncome_Euro_min} - {MonthlyIncome_Euro_max}"
    MonthlyIncome_Euro_ranges.append(MonthlyIncome_Euro_range)

print(MonthlyIncome_Euro_ranges)

Data la distribuzione dei valori di 'MonthlyIncome_Euro' molto eterogenea all'interno di ciascun cluster, è possibile osservare intervalli non continui. Questo potrebbe essere il risultato di una variazione significativa nel reddito mensile dei dipendenti all'interno di ciascun cluster.

In [ ]:
# Visualizza la distribuzione di 'MonthlyIncome_Euro' per ciascun cluster
plt.figure(figsize=(8, 6))
sns.boxplot(x='Cluster', y='MonthlyIncome_Euro', data=hr_num)
plt.xlabel('Cluster')
plt.ylabel('MonthlyIncome_Euro')
plt.title('Distribuzione di MonthlyIncome_Euro per Cluster')
plt.show()

Ogni box rappresenta la distribuzione dei valori all'interno di un cluster, con la linea orizzontale nel mezzo che indica la mediana, la parte inferiore del rettangolo che indica il primo quartile (25%) e la parte superiore del rettangolo che indica il terzo quartile (75%). È evidente un "salto" di reddito che giustifica la discontinuità degli intervalli ottenuti.

In [ ]:
# Definisci gli estremi di MonthlyIncome_Euro (inclusi -> right=True)
intervalli_eta = [(110, 665), (666, 1400), (1401, 2200)]

hr_num['MonthlyIncome_Euro'] = pd.cut(hr_num['MonthlyIncome_Euro'], bins=[intervallo[0] for intervallo in intervalli_eta] + [intervalli_eta[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_eta], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='MonthlyIncome_Euro', data=hr_num, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['MonthlyIncome_Euro'], attrition_no['MonthlyIncome_Euro'])

# Stampa il risultato del test
alpha = 0.01
if p_value < alpha:
    print("La differenza nel reddito mensile tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel reddito mensile tra i gruppi di Attrition.")

Dall'osservazione degli istogrammi e del violin plot sembra che non ci sia differenza tra i gruppi per Attrition = 'Yes' e 'No'.
Il t-test invece riporta un risultato differente. È possibile quindi che il reddito mensile non sia molto significativo. Infatti, eseguendo il t-test con un alpha = 0.01 si ottiene che non vi è evidenza di una differenza significativa nel reddito mensile tra i gruppi di Attrition.

## Attrition & NumCompaniesWorked

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'Yes'
training_counts_yes = attrition_yes['NumCompaniesWorked'].value_counts().sort_index()

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'No'
training_counts_no = attrition_no['NumCompaniesWorked'].value_counts().sort_index()

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo grafico a barre per Attrition = 'Yes'
axes[0].bar(training_counts_yes.index, training_counts_yes.values)
axes[0].set_title('Distribuzione NumCompaniesWorked - Attrition = Yes')
axes[0].set_xlabel('NumCompaniesWorked')
axes[0].set_ylabel('Frequency')

# Crea il secondo grafico a barre per Attrition = 'No'
axes[1].bar(training_counts_no.index, training_counts_no.values)
axes[1].set_title('Distribuzione NumCompaniesWorked - Attrition = No')
axes[1].set_xlabel('NumCompaniesWorked')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.tight_layout()

# Visualizza i grafici affiancati
plt.show()


In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='NumCompaniesWorked', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['NumCompaniesWorked'].mean()
stds = hr.groupby('Attrition')['NumCompaniesWorked'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -3, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -3.5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Attraverso il violin plot, al confronto tra i due gruppi della distribuzione del numero di compagnie in cui ogni dipendente ha lavorato in precedenza, la parte più ampia del violino per i dipendenti con Attrition = Yes è spostata verso il basso rispetto a quella per i dipendenti con Attrition = No, seppure anche quest'ultima si concentra tra i 0 e 4 numero di compagnie; ciò potrebbe indicare che i dipendenti che hanno lavorato in meno compagnie tendono a lasciare l'azienda più facilmente.
Si nota che per i dipendenti con Attrition = Yes la mediana è rappresentata in prossimità del primo quartile, ad indicare una maggiora affluenza di dipendenti con NumCompaniesWorked compreso tra 0 e 2.

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='NumCompaniesWorked', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['NumCompaniesWorked'], attrition_no['NumCompaniesWorked'])

# Stampa il risultato del test
alpha = 0.05
if p_value < alpha:
    print("La differenza nel NumCompaniesWorked tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel NumCompaniesWorked tra i gruppi di Attrition.")

Il t-test conferma quanto asssunto dall'osservazione del violin plot.

## Attritioin & PercentSalaryHike

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo istogramma per Attrition = 'Yes'
axes[0].hist(attrition_yes['PercentSalaryHike'], bins=10, edgecolor='black', alpha=0.5)
axes[0].set_title('Distribuzione PercentSalaryHike - Attrition = Yes')
axes[0].set_xlabel('PercentSalaryHike')
axes[0].set_ylabel('Frequency')

# Crea il secondo istogramma per Attrition = 'No'
axes[1].hist(attrition_no['PercentSalaryHike'], bins=10, edgecolor='black', alpha=0.5)
axes[1].set_title('Distribuzione PercentSalaryHike - Attrition = No')
axes[1].set_xlabel('PercentSalaryHike')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.tight_layout()

# Visualizza i grafici affiancati
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='PercentSalaryHike', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['PercentSalaryHike'].mean()
stds = hr.groupby('Attrition')['PercentSalaryHike'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, 6.5, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, 5.5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Attraverso il violin plot, al confronto tra i due gruppi della distribuzione dell'aumento in percentuale del salario rispetto l'anno precedente, la parte più ampia del violino per i dipendenti con Attrition = Yes è simile rispetto a quella per i dipendenti con Attrition = No, ciò potrebbe indicare che non ci sia tanta differenza tra i due gruppi.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['PercentSalaryHike'], attrition_no['PercentSalaryHike'])

# Stampa il risultato del test
alpha = 0.01
if p_value < alpha:
    print("La differenza nel PercentSalaryHike tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel PercentSalaryHike tra i gruppi di Attrition.")

Anche dall'osservazione degli istogrammi sembra che non ci sia differenza tra i gruppi per Attrition = 'Yes' e 'No'.
Il t-test invece riporta un risultato differente. È possibile quindi che l'aumento percentuale del salario rispetto all'anno precedente non sia molto significativo. Infatti, eseguendo il t-test con un alpha = 0.01 si ottiene che non vi è evidenza di una differenza significativa.

## Attrition & TotalWorkingYears

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='TotalWorkingYears', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['TotalWorkingYears'].mean()
stds = hr.groupby('Attrition')['TotalWorkingYears'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -10, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -12, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Al confronto tra i due gruppi della distribuzione del numero totale di anni lavorati fino adesso da ogni dipendente, la parte più ampia del violino per i dipendenti con Attrition = Yes è spostata verso il basso come quella per i dipendenti con Attrition = No ma più ampia. Questo potrebbe indicare che in generale vi è un accumulo di dipendenti con non più di 10 anni di lavoro alle spalle e che i dipendenti che hanno lavorato meno anni tendono a lasciare l'azienda più facilmente.

In [ ]:
# Estrai le colonne Attrition e TotalWorkingYears dal DataFrame hr_num
X = hr_num[['Attrition', 'TotalWorkingYears']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 16):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 16), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

In [ ]:
# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=4, random_state=42)

# Addestra il modello sui dati TotalWorkingYears
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi degli intervalli di TotalWorkingYears per ogni cluster
twy_ranges = []
for cluster_id in range(4):
    twy_values = hr.loc[hr['Cluster'] == cluster_id, 'TotalWorkingYears']
    twy_min = twy_values.min()
    twy_max = twy_values.max()
    twy_range = f"{twy_min} - {twy_max}"
    twy_ranges.append(twy_range)

print(twy_ranges)

In [ ]:
# Definisci gli estremi degli intervalli di TotalWorkingYears (inclusi -> right=True)
intervalli_twy = [(0, 7), (8, 14), (15,24), (25,40)]

hr['TWY_Intervallo'] = pd.cut(hr['TotalWorkingYears'], bins=[intervallo[0] for intervallo in intervalli_twy] + [intervalli_twy[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_twy], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='TWY_Intervallo', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto alle classi di TotalWorkingYears, la percentuale dei dipendenti che ha lasciato lo scorso anno è minore rispetto alla percentuale di chi è rimasto.

- Il 23% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 0 ed 7 anni di lavoro.
- Il 12% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di hi ha alle spalle tra 8 e 14 anni di lavoro.
- Il 09% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 15 e 24 anni di lavoro.
- Il 07% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 25 e 40 anni di lavoro.

In [ ]:
# Calcola la tabella di frequenza percentuali per colonna
table = pd.crosstab(hr['TWY_Intervallo'], hr['Attrition'], normalize='columns') * 100

# Stampa la tabella di frequenza
print(table)

A ulteriore conferma di quanto visto dai grafici si veda la tabella a doppia entrata. Questa risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 61% ha alle spalle tra 0 e 7 anni di lavoro. La seconda percentuale più alta, il 25%, è di chi ha alle spalle tra 8 e 14 anni di lavoro.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['TotalWorkingYears'], attrition_no['TotalWorkingYears'])

# Stampa il risultato del test
alpha = 0.05
if p_value < alpha:
    print("La differenza nel TotalWorkingYears tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel TotalWorkingYears tra i gruppi di Attrition.")

Anche il risutato che si ottiene dal t-test è in linea con quanto osservato in precedenza; sia con alpha = 0.05 che con alpha = 0.01

## Attrition & TrainingTimesLastYear

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='TrainingTimesLastYear', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['TrainingTimesLastYear'].mean()
stds = hr.groupby('Attrition')['TrainingTimesLastYear'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -1.6, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -1.9, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Al confronto tra i due gruppi della distribuzione del numero di volte in cui il singolo dipendente ha svolto formazione durante l'anno precedente, sia per i dipendenti con Attrition = Yes che per i dipendenti con Attrition = No si nota pressapoco la stessa distribuzione, ciò potrebbe indicare che non c'è differenza tra i due gruppi.

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='TrainingTimesLastYear', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Osservando il grafico a barre si riscontra che, per ogni ripartizione rispetto alle classi di TrainingTimesWorking, la percentuale dei dipendenti che ha lasciato lo scorso anno è minore rispetto alla percentuale di chi è rimasto.

- Il 19% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi lo scorso anno non ha svolto la formazione.
- Il 18% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi lo scorso anno ha svolto 3 volte la formazione.
- Il 17% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi lo scorso anno ha svolto 2 volte la formazione.
- il 14% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi lo scorso anno ha svolto sia 1 volta che 5 volte la formazione.
- il 06% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi lo scorso anno ha svolto 6 volte la formazione.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['TrainingTimesLastYear'], attrition_no['TrainingTimesLastYear'])

# Stampa il risultato del test
alpha = 0.01
if p_value < alpha:
    print("La differenza nel TrainingTimesLastYear tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel TrainingTimesLastYear tra i gruppi di Attrition.")

Anche se le distribuzioni dei valori sembrano simili nel violin plot, il t-test considera anche le differenze nelle medie dei due gruppi e rileva differenze statisticamente significative che non sono immediatamente evidenti solo guardando il grafico.

## Attrition & YearsAtCompany

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='YearsAtCompany', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['YearsAtCompany'].mean()
stds = hr.groupby('Attrition')['YearsAtCompany'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -9.5, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -11.5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Al confronto tra i due gruppi della distribuzione del numero di anni del singolo dipendente trascorsi nella compagnia, la parte più ampia del violino per i dipendenti con Attrition = Yes è spostata verso il basso: come quella per i dipendenti con Attrition = No ma più ampia. Questo potrebbe indicare che in generale vi è un accumulo di dipendenti con non più di 10 anni di presenza nella compagnia e che i dipendenti che hanno lavorato meno anni tendono a lasciare l'azienda più facilmente. Stesse conclusioni di quanto visto con la variabile TotalWorkingYears

In [ ]:
# Estrai le colonne Attrition e YearsAtCompany dal DataFrame hr_num
X = hr_num[['Attrition', 'YearsAtCompany']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 16):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 16), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

In [ ]:
# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=4, random_state=42)

# Addestra il modello sui dati YearsAtCompany
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi degli intervalli di YearsAtCompany per ogni cluster
twy_ranges = []
for cluster_id in range(4):
    twy_values = hr.loc[hr['Cluster'] == cluster_id, 'YearsAtCompany']
    twy_min = twy_values.min()
    twy_max = twy_values.max()
    twy_range = f"{twy_min} - {twy_max}"
    twy_ranges.append(twy_range)

print(twy_ranges)

In [ ]:
# Definisci gli estremi degli intervalli di YearsAtCompany (inclusi -> right=True)
intervalli_yac = [(0, 5), (6, 13), (14,24), (25,40)]

hr['YaC_Intervallo'] = pd.cut(hr['YearsAtCompany'], bins=[intervallo[0] for intervallo in intervalli_yac] + [intervalli_yac[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_yac], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='YaC_Intervallo', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto alle classi di YearsAtCompany, la percentuale dei dipendenti che ha lasciato lo scorso anno è minore rispetto alla percentuale di chi è rimasto.

- Il 19% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 0 ed 5 anni di lavoro.
- Il 16% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 25 e 40 anni di lavoro.
- L' 11% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di hi ha alle spalle tra 6 e 13 anni di lavoro.
- Il 08% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha alle spalle tra 14 e 24 anni di lavoro.

In [ ]:
# Calcola la tabella di frequenza percentuali per colonna
table = pd.crosstab(hr['YaC_Intervallo'], hr['Attrition'], normalize='columns') * 100

# Stampa la tabella di frequenza
print(table)

A ulteriore conferma di quanto visto dai grafici si veda la tabella a doppia entrata. Questa risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 70% ha alle spalle tra 0 e 5 anni di lavoro con la compagnia. La seconda percentuale più alta, il 24%, è di chi ha alle spalle tra 5 e 13 anni di lavoro con la compagnia.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['YearsAtCompany'], attrition_no['YearsAtCompany'])

# Stampa il risultato del test
alpha = 0.05
if p_value < alpha:
    print("La differenza nel YearsAtCompany tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel YearsAtCompany tra i gruppi di Attrition.")

Anche il risutato che si ottiene dal t-test è in linea con quanto osservato in precedenza; sia con alpha = 0.05 che con alpha = 0.01

## Attrition & YearsSinceLastPromotion

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='YearsSinceLastPromotion', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['YearsSinceLastPromotion'].mean()
stds = hr.groupby('Attrition')['YearsSinceLastPromotion'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -4.2, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Al confronto tra i due gruppi della distribuzione del numero di anni del singolo dipendente dall'ultima promozione, la parte più ampia del violino per i dipendenti con Attrition = No è spostata verso il basso: come quella per i dipendenti con Attrition = Yes ma leggermente più ampia e schiacciata intorno ai valori tra 0 e 2 anni. Questo potrebbe indicare che in generale vi è un accumulo di dipendenti con non più di 2.5 anni dall'ultima promozione e che i dipendenti che non hanno avuto una promozione entro i primi 2 anni tendono a lasciare l'azienda più facilmente.

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'Yes'
training_counts_yes = attrition_yes['YearsSinceLastPromotion'].value_counts().sort_index()

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'No'
training_counts_no = attrition_no['YearsSinceLastPromotion'].value_counts().sort_index()

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo grafico a barre per Attrition = 'Yes'
axes[0].bar(training_counts_yes.index, training_counts_yes.values)
axes[0].set_title('Distribuzione degli anni dall\'ultima promozione - Attrition = Yes')
axes[0].set_xlabel('YearsSinceLastPromotion')
axes[0].set_ylabel('Frequency')

# Crea il secondo grafico a barre per Attrition = 'No'
axes[1].bar(training_counts_no.index, training_counts_no.values)
axes[1].set_title('Distribuzione degli anni dall\'ultima promozione - Attrition = No')
axes[1].set_xlabel('YearsSinceLastPromotion')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.subplots_adjust(wspace=1.5)

# Visualizza i grafici affiancati
plt.show()

In [ ]:
# Estrai le colonne Attrition e YearsAtCompany dal DataFrame hr_num
X = hr_num[['Attrition', 'YearsSinceLastPromotion']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 10), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

In [ ]:
# Definizione delle features per addestrare il modello K-means
X = hr[['YearsSinceLastPromotion']].copy()  # Copia la colonna in un DataFrame

# Sostituisci i valori NaN con 0 nella colonna 'YearsSinceLastPromotion'
X['YearsSinceLastPromotion'].fillna(0, inplace=True)

# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=3, random_state=42)

# Addestra il modello sui dati YearsSinceLastPromotion
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi degli intervalli di YearsSinceLastPromotion per ogni cluster
yslp_ranges = []
for cluster_id in range(3):
    yslp_values = hr.loc[hr['Cluster'] == cluster_id, 'YearsSinceLastPromotion']
    yslp_min = yslp_values.min()
    yslp_max = yslp_values.max()
    yslp_range = f"{yslp_min} - {yslp_max}"
    yslp_ranges.append(yslp_range)

print(yslp_ranges)

In [ ]:
# Definisci gli estremi degli intervalli di YearsSinceLastPromotion (inclusi -> right=True)
intervalli_yslp = [(0, 3), (4, 8), (9,15)]

hr['YsLp_Intervallo'] = pd.cut(hr['YearsSinceLastPromotion'], bins=[intervallo[0] for intervallo in intervalli_yslp] + [intervalli_yslp[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_yslp], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='YsLp_Intervallo', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Dai grafici a barre si riscontra che, per ogni ripartizione rispetto alle classi di YearsSinceLastPromotion, la percentuale dei dipendenti che ha lasciato lo scorso anno è minore rispetto alla percentuale di chi è rimasto.

- Il 15% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha ricevuto una promozione tra gli ultimi 4-8 anni di lavoro.
- Il 14% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha ricevuto una promozione tra 0-3 anni di lavoro.
- L' 12% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha ricevuto una promozione tra gli ultimi 9-15 anni di lavoro.

In [ ]:
# Calcola la tabella di frequenza percentuali per colonna
table = pd.crosstab(hr['YsLp_Intervallo'], hr['Attrition'], normalize='columns') * 100

# Stampa la tabella di frequenza
print(table)

A conferma di quanto visto dai grafici si veda la tabella a doppia entrata. Questa risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 71% ha avuto una promozione negli ultimi 3 anni. La seconda percentuale più alta, il 22%, è di chi ha avuto una promozione tra gli ultimi 4-8 anni di lavoro.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['YearsSinceLastPromotion'], attrition_no['YearsSinceLastPromotion'])

# Stampa il risultato del test
alpha = 0.01
if p_value < alpha:
    print("La differenza nel YearsSinceLastPromotion tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel YearsSinceLastPromotion tra i gruppi di Attrition.")

A dimostrazione del fatto che non c'è molta differenza tra i due gruppi, Attrition = Yes e Attrition = No, il t-test riporta che la differenza è statisticamente significativa per un alpha = 0.05; con un alpha = 0.01 il t-test riporta che non vi è evidenza di una differenza significativa nella variabile tra i due gruppi di Attrition.

## Attrition & YearsWithCurrManager

In [ ]:
plt.figure(figsize=(8, 8))

# Violin plot
ax = sns.violinplot(y='YearsWithCurrManager', x='Attrition', data=hr, inner='box')

# Calcola media e deviazione standard
means = hr.groupby('Attrition')['YearsWithCurrManager'].mean()
stds = hr.groupby('Attrition')['YearsWithCurrManager'].std()

# Colore per media e deviazione standard
colors = ['black', 'black']

for i, attrition in enumerate(means.index):
    ax.text(i, -4.2, f'Mean: {means.values[i]:.2f}', ha='center', color=colors[i])
    ax.text(i, -5, f'Std: {stds.values[i]:.2f}', ha='center', color=colors[i])
    ax.scatter([], [], color=colors[i])

plt.show()

Al confronto tra i due gruppi della distribuzione del numero di anni del singolo dipendente trascorsi con il manager corrente, la parte più ampia del violino per i dipendenti con Attrition = Yes è spostata verso il basso: come quella per i dipendenti con Attrition = No ma più ampia. Questo potrebbe indicare che in generale vi è un accumulo di dipendenti con non più di 5 anni con il manager corrente e che i dipendenti che hanno lavorato meno anni tendono a lasciare l'azienda più facilmente. Stesse conclusioni di quanto visto con la variabile TotalWorkingYears

In [ ]:
# Filtra i dati per Attrition = 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'Yes'
training_counts_yes = attrition_yes['YearsWithCurrManager'].value_counts().sort_index()

# Calcola la frequenza dei valori di TrainingTimesLastYear per Attrition = 'No'
training_counts_no = attrition_no['YearsWithCurrManager'].value_counts().sort_index()

# Crea una griglia di plot con 1 riga e 2 colonne
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Crea il primo grafico a barre per Attrition = 'Yes'
axes[0].bar(training_counts_yes.index, training_counts_yes.values)
axes[0].set_title('Distribuzione degli anni con il manager corrente - Attrition = Yes')
axes[0].set_xlabel('YearsWithCurrManager')
axes[0].set_ylabel('Frequency')

# Crea il secondo grafico a barre per Attrition = 'No'
axes[1].bar(training_counts_no.index, training_counts_no.values)
axes[1].set_title('Distribuzione degli anni con il manager corrente - Attrition = No')
axes[1].set_xlabel('YearsWithCurrManager')
axes[1].set_ylabel('Frequency')

# Regola lo spaziamento tra i grafici
plt.subplots_adjust(wspace=1.5)

# Visualizza i grafici affiancati
plt.show()

Già da questo primo grafico a barre i possono notare delle differenze tra i due gruppi di Attrition

In [ ]:
# Estrai le colonne Attrition e YearsWithCurrManager dal DataFrame hr_num
X = hr_num[['Attrition', 'YearsWithCurrManager']].values

# Inizializza una lista per salvare le inertia per ogni valore di n_clusters
inertia_values = []

# Prova diversi valori di n_clusters e calcola l'inertia per ognuno
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia_values.append(kmeans.inertia_)

# Plotta il grafico del metodo del gomito
plt.plot(range(1, 15), inertia_values, marker='o')
plt.xlabel('Numero di Clusters')
plt.ylabel('Inertia')
plt.title('Metodo del Gomito')
plt.show()

In [ ]:
# Definizione delle features per addestrare il modello K-means
# X = hr[['YearsSinceLastPromotion']].copy()  # Copia la colonna in un DataFrame

# Sostituisci i valori NaN con 0 nella colonna 'YearsSinceLastPromotion'
# X['YearsSinceLastPromotion'].fillna(0, inplace=True)

# Crea l'istanza del modello K-means
kmeans = KMeans(n_clusters=3, random_state=42)

# Addestra il modello sui dati YearsSinceLastPromotion
kmeans.fit(X)

# Assegna le etichette di cluster ai dipendenti
hr['Cluster'] = kmeans.labels_

# Calcola i valori minimi e massimi degli intervalli di YearsSinceLastPromotion per ogni cluster
ywcm_ranges = []
for cluster_id in range(3):
    ywcm_values = hr.loc[hr['Cluster'] == cluster_id, 'YearsWithCurrManager']
    ywcm_min = ywcm_values.min()
    ywcm_max = ywcm_values.max()
    ywcm_range = f"{ywcm_min} - {ywcm_max}"
    ywcm_ranges.append(ywcm_range)

print(ywcm_ranges)

In [ ]:
# Definisci gli estremi degli intervalli di YearsSinceLastPromotion (inclusi -> right=True)
intervalli_ywcm = [(0, 4), (5, 9), (10,17)]

hr['YwCm_Intervallo'] = pd.cut(hr['YearsWithCurrManager'], bins=[intervallo[0] for intervallo in intervalli_ywcm] + [intervalli_ywcm[-1][1] + 1],
                        labels=[f"{intervallo[0]}-{intervallo[1]}" for intervallo in intervalli_ywcm], right=True)

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='YwCm_Intervallo', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

Dai grafici a barre si riscontra che, per ogni ripartizione rispetto alle classi di YearsSinceLastPromotion, la percentuale dei dipendenti che ha lasciato lo scorso anno è minore rispetto alla percentuale di chi è rimasto.

- Il 14% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha trascorso al massimo 4 anni con il manager corrente.
- Il 12% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha trascorso tra 5-9 anni con il manager corrente.
- Lo 04% di chi ha abbandonato l'azienda lo scorso anno rientra nella classe di chi ha trascorso tra 10-17 anni con il manager corrente.

In [ ]:
# Calcola la tabella di frequenza percentuali per colonna
table = pd.crosstab(hr['YwCm_Intervallo'], hr['Attrition'], normalize='columns') * 100

# Stampa la tabella di frequenza
print(table)

A conferma di quanto visto dai grafici si veda la tabella a doppia entrata. Questa risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 63% ha trascorso non più di 4 anni con il manager corrente. La seconda percentuale più alta, il 36%, è di chi ha trascorso con il manager corrente tra i 5 ed i 9 anni.

In [ ]:
# Filtra i dati per Attrition 'Yes' e 'No'
attrition_yes = hr[hr['Attrition'] == 'Yes']
attrition_no = hr[hr['Attrition'] == 'No']

# Esegui il test t indipendente tra i due gruppi
t_statistic, p_value = stats.ttest_ind(attrition_yes['YearsWithCurrManager'], attrition_no['YearsWithCurrManager'])

# Stampa il risultato del test
alpha = 0.05
if p_value < alpha:
    print("La differenza nel YearsWithCurrManager tra i gruppi di Attrition è statisticamente significativa.")
else:
    print("Non vi è evidenza di una differenza significativa nel YearsWithCurrManager tra i gruppi di Attrition.")

Anche il risutato che si ottiene dal t-test è in linea con quanto osservato in precedenza; sia con alpha = 0.05 che con alpha = 0.01 la differenza negli anni con il manager corrente tra i gruppi di Attrition è statisticamente significativa.

## Matrice di contingenza dei dati categoriali

In [ ]:
# Dataframe con soli variabili categoriali
hr_cat = hr_var_dec[['Attrition', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance',
                    'BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']]

In [ ]:
hr_cat = hr_cat.astype('category')

In [ ]:
# Creare la tabella di contingenza
contingency_table = pd.crosstab(index=hr_cat['Attrition'],
                               columns=[hr_cat['JobInvolvement'], hr_cat['PerformanceRating'],
                                        hr_cat['EnvironmentSatisfaction'], hr_cat['JobSatisfaction'],
                                        hr_cat['WorkLifeBalance'], hr_cat['BusinessTravel'], hr_cat['Department'],
                                        hr_cat['Education'], hr_cat['EducationField'], hr_cat['Gender'],
                                        hr_cat['JobRole'], hr_cat['MaritalStatus']])

In [ ]:
# Frequenza delle combinazioni per Attrition = Yes
freq_yes = contingency_table.loc['Yes']
max_freq_yes_comb = freq_yes.idxmax()
max_freq_yes_count = freq_yes.max()

print("Combinazione con la frequenza più alta per Attrition = Yes:")
print(max_freq_yes_comb)
print("Frequenza:", max_freq_yes_count)

# Frequenza delle combinazioni per Attrition = No
freq_no = contingency_table.loc['No']
max_freq_no_comb = freq_no.idxmax()
max_freq_no_count = freq_no.max()

print("\nCombinazione con la frequenza più alta per Attrition = No:")
print(max_freq_no_comb)
print("Frequenza:", max_freq_no_count)

La tabella di contingenza calcola la frequenza delle combinazioni di valori per le variabili specificate. Ogni cella della tabella rappresenta il numero di osservazioni che corrispondono a una specifica combinazione di valori.
L'output che si è ottenuto indica le combinazioni di valori con la frequenza più alta per `Attrition = Yes` e `Attrition = No`. Si può vedere che le combinazioni di valori sono presentate come tuple, con ogni elemento che rappresenta i valori delle rispettive variabili categoriche.

Per `Attrition = Yes`, la combinazione con la frequenza più alta è:
- JobInvolvement: 'High'
- PerformanceRating: 'Excellent'
- EnvironmentSatisfaction: 'High'
- JobSatisfaction: 'High'
- WorkLifeBalance: 'Bad'
- BusinessTravel: 'Travel_Frequently'
- Department: 'Research & Development'
- Education: 'Bachelor'
- EducationField: 'Medical'
- Gender: 'Female'
- JobRole: 'Sales Executive'
- MaritalStatus: 'Married'

La frequenza associata a questa combinazione è 3.

Per `Attrition = No`, la combinazione con la frequenza più alta è:
- JobInvolvement: 'High'
- PerformanceRating: 'Excellent'
- EnvironmentSatisfaction: 'High'
- JobSatisfaction: 'High'
- WorkLifeBalance: 'Better'
- BusinessTravel: 'Travel_Rarely'
- Department: 'Sales'
- Education: 'Bachelor'
- EducationField: 'Medical'
- Gender: 'Female'
- JobRole: 'Laboratory Technician'
- MaritalStatus: 'Married'

La frequenza associata a questa combinazione è 6.

Queste informazioni forniscono una comprensione delle combinazioni di valori che sono più frequenti per le diverse categorie di `Attrition`.

Ha senso approfondire l'analisi esplorativa dei dati concentrandosi sulle variabili categoriali che risultano presenti nelle combinazioni di valori più frequenti. Questo può essere utile per identificare le caratteristiche più rilevanti o significative che potrebbero influenzare l'attrition.

Concentrarsi sulle combinazioni di valori più frequenti permette di individuare i modelli o le relazioni che potrebbero avere un impatto significativo sull'attrition. Si può esaminare come queste variabili categoriali influenzano l'attrition e determinare se esistono correlazioni o tendenze evidenti.

In [ ]:
from scipy.stats import chi2_contingency

# Lista delle variabili esplicative categoriche
variabili_esplicative = ['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction',
                         'WorkLifeBalance', 'BusinessTravel', 'Department', 'Education', 'EducationField',
                         'Gender', 'JobRole', 'MaritalStatus']

# Lista delle variabili esplicative significative
variabili_significative = []

# Calcolo del test del chi-quadro per ogni variabile esplicativa
for variabile in variabili_esplicative:
    contingency_table = pd.crosstab(index=hr_var_dec['Attrition'], columns=hr_var_dec[variabile])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)

    # Controllo del valore di p del test del chi-quadro
    if p_value < 0.05:  # Impostare il livello di significatività desiderato
        variabili_significative.append(variabile)

# Output delle variabili esplicative significative
if len(variabili_significative) > 0:
    print("Le seguenti variabili esplicative categoriche risultano significative:")
    for variabile in variabili_significative:
        print(variabile)
else:
    print("Nessuna variabile esplicativa categorica risulta significativa.")


In [ ]:
# Lista delle variabili esplicative categoriche
variabili_esplicative = ['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction',
                         'WorkLifeBalance', 'BusinessTravel', 'Department', 'Education', 'EducationField',
                         'Gender', 'JobRole', 'MaritalStatus']

# Lista delle variabili esplicative significative
variabili_significative = []

# Calcolo del test del chi-quadro per ogni variabile esplicativa
for variabile in variabili_esplicative:
    contingency_table = pd.crosstab(index=hr_var_dec['Attrition'], columns=hr_var_dec[variabile])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)

    # Controllo del valore di p del test del chi-quadro
    if p_value < 0.01:  # Impostare il livello di significatività desiderato
        variabili_significative.append(variabile)

# Output delle variabili esplicative significative
if len(variabili_significative) > 0:
    print("Le seguenti variabili esplicative categoriche risultano significative:")
    for variabile in variabili_significative:
        print(variabile)
else:
    print("Nessuna variabile esplicativa categorica risulta significativa.")

In [ ]:
hr_cat2 = hr_cat.copy()
# Converti la variabile Attrition in numerica e formatta le altre in category
hr_cat2['Attrition'] = hr_cat2['Attrition'].map({'Yes': 1, 'No': 0})
hr_cat2['Attrition'] = hr_cat2['Attrition'].astype(int)

In [ ]:
# Lista delle variabili esplicative categoriche
variabili_esplicative = ['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction',
                         'WorkLifeBalance', 'BusinessTravel', 'Department', 'Education', 'EducationField',
                         'Gender', 'JobRole', 'MaritalStatus']

# Lista delle variabili esplicative significative
variabili_significative = []

# Calcolo dell'ANOVA bidirezionale per ogni variabile esplicativa
for variabile in variabili_esplicative:
    formula = 'Attrition ~ ' + variabile
    model = ols(formula, data=hr_cat2).fit()
    table = sm.stats.anova_lm(model, typ=2)
    p_value = table.loc[variabile, 'PR(>F)']

    if p_value < 0.05:  # Impostare il livello di significatività desiderato
        variabili_significative.append(variabile)

# Generazione dell'output con le variabili esplicative significative
if len(variabili_significative) > 0:
    output = "Le seguenti variabili esplicative categoriche risultano significative: " + ", ".join(variabili_significative)
else:
    output = "Nessuna variabile esplicativa categorica risulta significativa."

print(output)

In [ ]:
# Calcolo dell'ANOVA bidirezionale per ogni variabile esplicativa con alpha = 0.01
for variabile in variabili_esplicative:
    formula = 'Attrition ~ ' + variabile
    model = ols(formula, data=hr_cat2).fit()
    table = sm.stats.anova_lm(model, typ=2)
    p_value = table.loc[variabile, 'PR(>F)']

    if p_value < 0.01:  # Impostare il livello di significatività desiderato
        variabili_significative.append(variabile)

# Generazione dell'output con le variabili esplicative significative
if len(variabili_significative) > 0:
    output = "Le seguenti variabili esplicative categoriche risultano significative: " + ", ".join(variabili_significative)
else:
    output = "Nessuna variabile esplicativa categorica risulta significativa."

print(output)

Dai test del chi-quadro e ANOVA si è ottenuta una lista di variabili esplicative categoriche che risultano significative e che si vedranno in dettaglio.

## Attrition & JobInvolvement

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='JobInvolvement', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['JobInvolvement'], hr['Attrition'], normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di JobInvolvement
order = ['Low', 'Medium', 'High', 'Very High']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "JobInvolvement", la percentuale dei dipendenti che ha lasciato lo scorso anno è molto minore rispetto alla percentuale di chi è rimasto.

- Il 22% di chi ha abbandonato l'azienda lo scorso anno ha dato "Low" al livello di coinvolgimento lavorativo.
- Il 16% di chi ha abbandonato l'azienda lo scorso anno ha dato "Medium" al livello di coinvolgimento lavorativo.
- Il 15% di chi ha abbandonato l'azienda lo scorso anno ha dato "High" al livello di coinvolgimento lavorativo.
- Il 18% di chi ha abbandonato l'azienda lo scorso anno ha dato "Very High" al livello di coinvolgimento lavorativo.

Dalla tabella a doppia entrata risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 56% ha valutato il livello di coinvolgimento lavorativo come "High". La seconda percentuale più alta, il 25%, è di chi ha valutato il livello di coinvolgimento lavorativo come "Medium".

## Attrition & EnvironmentSatisfaction

In [ ]:
plt.figure(figsize=(8,10))
ax = sns.countplot(x='EnvironmentSatisfaction', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 30, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['EnvironmentSatisfaction'], hr['Attrition'], normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di EnvironmentSatisfaction
order = ['Low', 'Medium', 'High', 'Very High']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "EnvironmentSatisfaction", la percentuale dei dipendenti che ha lasciato lo scorso anno è molto minore rispetto alla percentuale di chi è rimasto.

- Il 25% di chi ha abbandonato l'azienda lo scorso anno ha dato "Low" al livello di soddisfazione dell'ambiente di lavoro.
- Il 15% di chi ha abbandonato l'azienda lo scorso anno ha dato "Medium" al livello di soddisfazione dell'ambiente di lavoro.
- Il 14% di chi ha abbandonato l'azienda lo scorso anno ha dato "High" al livello di soddisfazione dell'ambiente di lavoro.
- Il 13% di chi ha abbandonato l'azienda lo scorso anno ha dato "Very High" al livello di soddisfazione dell'ambiente di lavoro.

Dalla tabella a doppia entrata risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 30% ha valutato il livello di soddisfazione dell'ambiente di lavoro come "Low". La seconda percentuale più alta, il 27%, è di chi ha valutato il livello di soddisfazione dell'ambiente di lavoro come "High".

## Attrition & JobSatisfaction

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='JobSatisfaction', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['JobSatisfaction'], hr['Attrition'], normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di EnvironmentSatisfaction
order = ['Low', 'Medium', 'High', 'Very High']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "JobSatisfaction", la percentuale dei dipendenti che ha lasciato lo scorso anno è molto minore rispetto alla percentuale di chi è rimasto.

- Il 23% di chi ha abbandonato l'azienda lo scorso anno ha dato "Low" al livello di soddisfazione del lavoro.
- Il 16% di chi ha abbandonato l'azienda lo scorso anno ha dato "Medium" al livello di soddisfazione del lavoro.
- Il 16% di chi ha abbandonato l'azienda lo scorso anno ha dato "High" al livello di soddisfazione del lavoro.
- Il 11% di chi ha abbandonato l'azienda lo scorso anno ha dato "Very High" al livello di soddisfazione del lavoro.

Dalla tabella a doppia entrata risalta che, tra i dipendenti che hanno abbandonato lo scorso anno, il 31% ha valutato il livello di soddisfazione del lavoro come "High". La seconda percentuale più alta, il 28%, è di chi ha valutato il livello di soddisfazione del lavoro come "Low".

## Attrition & WorkLifeBalance

In [ ]:
# Si crea un grafico a barre che mostra il conteggio del bilanciamento tra lavoro e vita privata
# dei dipendenti di un'azienda, suddiviso in base all'attrition (la tendenza dei dipendenti a lasciare l'azienda).

plt.figure(figsize=(8,8))
ax = sns.countplot(x='WorkLifeBalance', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['WorkLifeBalance'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di WorkLifeBalance
order = ['Bad', 'Good', 'Better','Best']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "WorkLifeBalance", la percentuale dei dipendenti che ha lasciato l'anno precedente è molto minore rispetto a chi non ha lasciato.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, il 54% riteneva il bilanciamento tra lavoro e vita privata come "Better".
il 24% invece riteneva il bilanciamento lavoro-vita "Good".

Potrebbe quindi non essere una variabile significativa

## Attrition & BusinessTravel

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='BusinessTravel', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['BusinessTravel'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di BusinessTravel
order = ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "BusinessTravel", la percentuale dei dipendenti che ha lasciato l'anno precedente è molto minore rispetto a chi non ha lasciato.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, il 66% riteneva viaggiava raramente.
il 29% invece viaggiava per lavoro frequentemente.

## Attrition & Department

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='Department', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['Department'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di Department
order = ['Sales', 'Research & Development', 'Human Resources']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "Department", la percentuale dei dipendenti che ha lasciato l'anno precedente è molto minore rispetto a chi non ha lasciato. Unica eccezione nel settore 'Human Resources' dove i due valori della variabile 'Attrition' non sono molto distanti.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, il 64% proveniva dal settore Ricerca & Sviluppo dell'azienda.
il 28% invece dal reparto Vendite.

Questo risultato ha un riscontro nella realtà in quanto offerte di lavoro nel settore informatico sono in numero maggiore.

## Attrition & EducationField

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.countplot(x='EducationField', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['EducationField'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di EducationField
order = ['Life Sciences', 'Other', 'Medical', 'Marketing', 'Technical Degree', 'Human Resources']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "EducationField", la percentuale dei dipendenti che ha lasciato l'anno precedente è molto minore rispetto a chi non ha lasciato. Unica eccezione nel campo di educazione 'Human Resources' dove i due valori della variabile 'Attrition' quasi si eguagliano.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, il 42% ha studiato nel campo delle Scienze della Vita.
il 31% invece ha avuto una formazione Medica.

## Attrition & JobRole

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.countplot(x='JobRole', data=hr_var_dec, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['JobRole'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di JobRole
order = ['Healthcare Representative', 'Research Scientist', 'Sales Executive', 'Human Resources', 'Research Director', 'Laboratory Technician',
            'Manufacturing Director', 'Sales Representative', 'Manager']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "JobRole", la percentuale dei dipendenti che ha lasciato l'anno precedente è  minore rispetto a chi non ha lasciato.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, i primi tre ruoli lavorativi con maggiore percentuale sono:

1. Sales Executive con il 23%
2. Research Scientist con il 22%
3. Laboratory Technician con il 18%

Ciò riconferma quanto visto con le variabili 'EducationField' e 'Department'

## Attrition & MaritalStatus

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='MaritalStatus', data=hr, hue="Attrition")
ax.set_ylabel('# of Employee')
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

    ax.text(left.get_x() + left.get_width()/2., height_l + 40, '{0:.0%}'.format(height_l/total), ha="center")
    ax.text(right.get_x() + right.get_width()/2., height_r + 40, '{0:.0%}'.format(height_r/total), ha="center")

In [ ]:
# Crea la tabella a doppia entrata con valori percentuali per colonna
table = pd.crosstab(hr_var_dec['MaritalStatus'],hr['Attrition'],normalize='columns') * 100
# Definisci l'ordine desiderato per i valori di JobRole
order = ['Married', 'Single', 'Divorced']
# Ordina la tabella in base all'ordine definito
table = table.reindex(order)
# Visualizza la tabella
print(table)

Dal grafico a barre si riscontra che, per ogni ripartizione rispetto al "MaritalStatus", la percentuale dei dipendenti che ha lasciato l'anno precedente è minore rispetto a chi non ha lasciato.

Dalla tabella a doppia entrata viene risaltato invece che, tra i dipendenti che hanno lasciato l'anno precedente, il 50% erano single e il 35% sposati.

# Modello di regressione logistica

## Preparazione DataFrame

In [ ]:
# Si prova a vedere il modello completo

hr_logit = hr[['Nr Ore','JobInvolvement','PerformanceRating','EnvironmentSatisfaction','JobSatisfaction','WorkLifeBalance','Age',
'Attrition','BusinessTravel','Department','DistanceFromHome','Education','EducationField','Gender','JobLevel','JobRole','MaritalStatus',
'MonthlyIncome_Euro','NumCompaniesWorked','PercentSalaryHike','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
'YearsSinceLastPromotion','YearsWithCurrManager']]

In [ ]:
hr_logit['JobInvolvement'] = hr_logit['JobInvolvement'].map({1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'})
hr_logit['PerformanceRating'] = hr_logit['PerformanceRating'].map({1: 'Low', 2: 'Good', 3: 'Excellent', 4: 'Outstanding'})
hr_logit['EnvironmentSatisfaction'] = hr_logit['EnvironmentSatisfaction'].map({1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'})
hr_logit['JobSatisfaction'] = hr_logit['JobSatisfaction'].map({1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'})
hr_logit['WorkLifeBalance'] = hr_logit['WorkLifeBalance'].map({1: 'Bad', 2: 'Good', 3: 'Better', 4: 'Best'})
hr_logit['Education'] = hr_logit['Education'].map({1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'})
hr_logit['MonthlyIncome_Euro'] = hr_logit['MonthlyIncome_Euro'].astype('int64')

## Creazione variabili dummy

Si convertono le variabili categoriche in variabili dummy (o indicatrici/fittizie) che può essere facilmente utilizzato dagli algoritmi.

Le variabili categoriche rappresentano attributi che non possono essere misurati in termini numerici continui, ma piuttosto descrivono categorie o gruppi distinti.

Per utilizzare queste variabili all'interno di modelli, è necessario convertirle in un formato numerico. Le variabili dummy sono create assegnando un valore binario (0 o 1) a ciascuna categoria della variabile categorica. Viene creata una variabile fittizia separata per ogni categoria, dove il valore 1 indica che l'osservazione appartiene a quella categoria e il valore 0 indica che non appartiene.

Le variabili indicatrici consentono quindi di rappresentare informazioni categoriche in modo numerico, consentendo l'utilizzo di queste variabili in modelli che richiedono input numerici.

In [ ]:
# Creazione di una variabile fittizia per alcune delle variabili categoriche e eliminazione della prima (per evitare la duplicazione di informazioni)

dummy1 = pd.get_dummies(hr_logit[['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
                                 'JobSatisfaction', 'WorkLifeBalance','BusinessTravel', 'Department',
                                 'Education','EducationField', 'Gender', 'JobLevel', 'JobRole',
                                 'MaritalStatus']], drop_first=True).astype(int)

# Si aggiungono i risultati al dataframe
hr_logit_dummy = pd.concat([hr_logit, dummy1], axis=1)

In [ ]:
# Avendo creato le variabili dummy si possono rimuovere dal dataframe le seguenti colonne
hr_logit_dummy = hr_logit_dummy.drop(['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
                          'JobSatisfaction', 'WorkLifeBalance', 'BusinessTravel', 'Department',
                          'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole',
                          'MaritalStatus'], axis=1)

In [ ]:
# Si codifica la variabile risposta in 0 e 1
hr_logit_dummy['Attrition'] = hr_logit_dummy['Attrition'].replace({'Yes': 1, "No": 0})

## Modello completo di Regressione logistica

### Modello completo

In [ ]:
# Si seleziona le variabili esplicative
Xd = hr_logit_dummy.drop(['Attrition'], axis=1)

In [ ]:
# Si assegna a y la variabile risposta
Yd = hr_logit_dummy['Attrition']

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Xd[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike','StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
'DistanceFromHome','Age','Nr Ore']] = scaler.fit_transform(Xd[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'DistanceFromHome','Age','Nr Ore']])

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
Xd = sm.add_constant(Xd)

In [ ]:
import statsmodels.api as sm

# Stima del modello completo di regressione logistica
log_model = sm.GLM(Yd, Xd, family=sm.families.Binomial())
fit0 = log_model.fit()

# Stampa il riassunto del modello
print(fit0.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1559.1

# Numero di parametri nel modello
num_parametri = 53+1

# Numero di osservazioni nel dataset di addestramento
n = len(Yd)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Si calcola il BIC (Bayesian Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

 # Log-likelihood del modello
log_likelihood = fit0.llf

# Numero di parametri del modello (è uguale al numero di coefficienti nel modello + 1 per l'intercetta)
num_parametri = fit0.df_model + 1

# Numero di osservazioni nel set di dati
n = len(Yd)

# Calcolo il BIC
BIC = -2 * log_likelihood + num_parametri * np.log(n)

# Un valore BIC inferiore indica una migliore adeguabilità del modello ai dati, tenendo conto della complessità del modello.

BIC

Variabili quantitative considerate rilevanti dall'analisi esplorativa attraverso il T-test sono:

	Variabili con differenza significativa tra i gruppi di Attrition sia con alpha = 0.05 che con alpha = 0.01:
		1. Age
		2. NumCompaniesWorked
		3. TotalWorkingYears
		4. TrainingTimesLastYear
		5. YearsAtCompany
		6. YearsWithCurrManager

	Variabili con differenza significativa tra i gruppi di Attrition solo per alpha = 0.05:
		1. MonthlyIncome_Euro
		2. PercentSalaryHike
		3. YearsSinceLastPromotion

Variabili categoriali considerate rilevanti dall'analisi esplorativa attraverso il test del Chi-Quadro sono:

	Variabili con differenza significativa tra i gruppi di Attrition sia con alpha = 0.05 che con alpha = 0.01:
		1. EnvironmentSatisfaction
		2. JobSatisfaction
		3. WorkLifeBalance
		4. BusinessTravel
		5. Department
		6. EducationField
		7. JobRole
		8. MaritalStatus

	Variabili con differenza significativa tra i gruppi di Attrition solo per alpha = 0.05:
		1. JobInvolvement

### Si modella su variabili significative dall'EDA per alpha = 0.01

In [ ]:
hr_eda = hr_logit_dummy[['Attrition','Age','NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear',
            'YearsAtCompany','YearsWithCurrManager','EnvironmentSatisfaction_Low',
            'EnvironmentSatisfaction_Medium','EnvironmentSatisfaction_Very High','JobSatisfaction_Low',
            'JobSatisfaction_Medium','JobSatisfaction_Very High','WorkLifeBalance_Best',
            'WorkLifeBalance_Better','WorkLifeBalance_Good','BusinessTravel_Travel_Frequently',
            'BusinessTravel_Travel_Rarely','Department_Research & Development','Department_Sales',
            'EducationField_Life Sciences','EducationField_Marketing','EducationField_Medical','EducationField_Other',
            'EducationField_Technical Degree','JobRole_Human Resources','JobRole_Laboratory Technician','JobRole_Manager',
            'JobRole_Manufacturing Director','JobRole_Manufacturing Director','JobRole_Research Director','JobRole_Research Scientist',
            'JobRole_Sales Executive','JobRole_Sales Representative','MaritalStatus_Married','MaritalStatus_Single']]

In [ ]:
# Si codifica la variabile risposta in 0 e 1
hr_eda['Attrition'] = hr_eda['Attrition'].replace({'Yes': 1, "No": 0})

In [ ]:
Ye = hr_eda['Attrition']
Xe = hr_eda.drop('Attrition',axis=1)

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
scaler = StandardScaler()

Xe[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsWithCurrManager','Age']] = scaler.fit_transform(Xe[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
            'YearsWithCurrManager','Age']])

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
Xe = sm.add_constant(Xe)

In [ ]:
# Stima del modello di regressione logistica su variabili significative dall'EDA per alpha = 0.01
log_model_e = sm.GLM(Ye, Xe, family=sm.families.Binomial())
fit1 = log_model_e.fit()

# Stampa il riassunto del modello
print(fit1.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1602.8

# Numero di parametri nel modello
num_parametri = 34+1

# Numero di osservazioni nel dataset di addestramento
n = len(Ye)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Si calcola il BIC (Bayesian Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

 # Log-likelihood del modello
log_likelihood = fit1.llf

# Numero di parametri del modello (è uguale al numero di coefficienti nel modello + 1 per l'intercetta)
num_parametri = fit1.df_model + 1

# Numero di osservazioni nel set di dati
n = len(Ye)

# Calcolo il BIC
BIC = -2 * log_likelihood + num_parametri * np.log(n)

# Un valore BIC inferiore indica una migliore adeguabilità del modello ai dati, tenendo conto della complessità del modello.

BIC

In [ ]:
# Eseguo il test del rapporto di massima verosimiglianza
from scipy.stats import chi2
# Calcolo il log-likelihood dei due modelli
ll_full = fit0.llf
ll_reduced = fit1.llf

# Calcola la statistica del test del rapporto di massima verosimiglianza
lr_statistic = -2 * (ll_reduced- ll_full)

# Gradi di libertà
df = fit0.df_model - fit1.df_model

# Calcola il p-value usando la distribuzione chi-quadro
p_value = 1.0 - chi2.cdf(lr_statistic, df)

# Stampare i risultati del test
print("Likelihood Ratio Test:")
print("Test statistic:", lr_statistic)
print("P-value:", p_value)
print("Degrees of freedom:", df)

# Confronto con un valore di soglia, ad esempio 0.05
alpha = 0.05
if p_value < alpha:
    print("Il modello completo è preferibile al modello ridotto con le sole variabili significative all'1%.")
else:
    print("Non ci sono prove sufficienti per rigettare il modello ridotto con le sole variabili significative all'1%")

Dal confronto del modello completo con il modello con le sole variabili significative all'1% per l'EDA, si è riscontrato che sia per l'AICc che per il test del rapporto di verosimiglianza il modello completo è preferibile.
I BIC invece consiglia il modello ridotto ma solo perché pesa di più il numero minore di variabili usate.

### Modello di regressione logistica multivariabile – Modello su variabili significative dall’EDA per alpha = 0.05

In [ ]:
hr_eda1 = hr_logit_dummy[['Attrition','Age','NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear',
            'YearsAtCompany','YearsWithCurrManager','MonthlyIncome_Euro','PercentSalaryHike','YearsSinceLastPromotion','EnvironmentSatisfaction_Low',
            'EnvironmentSatisfaction_Medium','EnvironmentSatisfaction_Very High','JobSatisfaction_Low',
            'JobSatisfaction_Medium','JobSatisfaction_Very High','WorkLifeBalance_Best',
            'WorkLifeBalance_Better','WorkLifeBalance_Good','BusinessTravel_Travel_Frequently',
            'BusinessTravel_Travel_Rarely','Department_Research & Development','Department_Sales',
            'EducationField_Life Sciences','EducationField_Marketing','EducationField_Medical','EducationField_Other',
            'EducationField_Technical Degree','JobRole_Human Resources','JobRole_Laboratory Technician','JobRole_Manager',
            'JobRole_Manufacturing Director','JobRole_Manufacturing Director','JobRole_Research Director','JobRole_Research Scientist',
            'JobRole_Sales Executive','JobRole_Sales Representative','MaritalStatus_Married','MaritalStatus_Single','JobInvolvement_Low','JobInvolvement_Medium','JobInvolvement_Very High']]

In [ ]:
# Si codifica la variabile risposta in 0 e 1
hr_eda1['Attrition'] = hr_eda1['Attrition'].replace({'Yes': 1, "No": 0})

In [ ]:
Ye1 = hr_eda1['Attrition']
Xe1 = hr_eda1.drop('Attrition',axis=1)

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
scaler = StandardScaler()

Xe1[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsWithCurrManager','Age','MonthlyIncome_Euro','PercentSalaryHike','YearsSinceLastPromotion']] = scaler.fit_transform(Xe1[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
            'YearsWithCurrManager','Age','MonthlyIncome_Euro','PercentSalaryHike','YearsSinceLastPromotion']])

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
Xe1 = sm.add_constant(Xe1)

In [ ]:
# Stima del modello di regressione logistica su variabili significative dall'EDA per alpha = 0.05
log_model_e1 = sm.GLM(Ye1, Xe1, family=sm.families.Binomial())
fit2 = log_model_e1.fit()

# Stampa il riassunto del modello
print(fit2.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello modello ridotto con variabili significative al 5% per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1567.7

# Numero di parametri nel modello
num_parametri = 40+1

# Numero di osservazioni nel dataset di addestramento
n = len(Ye1)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Eseguo il test del rapporto di massima verosimiglianza tra modello completo e modello ridotto con variabili significative al 5%
from scipy.stats import chi2
# Calcolo il log-likelihood dei due modelli
ll_full = fit0.llf
ll_reduced = fit2.llf

# Calcola la statistica del test del rapporto di massima verosimiglianza
lr_statistic = -2 * (ll_reduced- ll_full)

# Gradi di libertà
df = fit0.df_model - fit2.df_model

# Calcola il p-value usando la distribuzione chi-quadro
p_value = 1.0 - chi2.cdf(lr_statistic, df)

# Stampare i risultati del test
print("Likelihood Ratio Test:")
print("Test statistic:", lr_statistic)
print("P-value:", p_value)
print("Degrees of freedom:", df)

# Confronto con un valore di soglia, ad esempio 0.05
alpha = 0.05
if p_value < alpha:
    print("Il modello completo è preferibile al modello ridotto con variabili significative al 5%.")
else:
    print("Non ci sono prove sufficienti per rigettare il modello ridotto con variabili significative al 5%.")

Il modello ridotto con variabili significative al 5% risulta, sia per il criterio AICc che per il test del rapporto di verosimiglianza, preferibile rispetto al modello completo.

### Costruzione modello con selezione delle variabili esplicative via metodo RFE" (Recursive Feature Elimination with Cross-Validation)

In [ ]:
import pandas as pd

EnvironmentSatisfaction = Xe1[['EnvironmentSatisfaction_Low','EnvironmentSatisfaction_Medium','EnvironmentSatisfaction_Very High']]
JobSatisfaction = Xe1[['JobSatisfaction_Low','JobSatisfaction_Medium','JobSatisfaction_Very High']]
WorkLifeBalance = Xe1[['WorkLifeBalance_Good','WorkLifeBalance_Better','WorkLifeBalance_Best']]
BusinessTravel = Xe1[['BusinessTravel_Travel_Frequently','BusinessTravel_Travel_Rarely']]
Department = Xe1[['Department_Research & Development','Department_Sales']]
EducationField = Xe1[['EducationField_Life Sciences','EducationField_Marketing','EducationField_Medical','EducationField_Other','EducationField_Technical Degree']]
JobRole = Xe1[['JobRole_Human Resources','JobRole_Laboratory Technician','JobRole_Manager','JobRole_Manufacturing Director','JobRole_Manufacturing Director','JobRole_Research Director','JobRole_Research Scientist','JobRole_Sales Executive','JobRole_Sales Representative']]
MaritalStatus = Xe1[['MaritalStatus_Married','MaritalStatus_Single']]
JobInvolvement = Xe1[['JobInvolvement_Low','JobInvolvement_Medium','JobInvolvement_Very High']]

dummy_vars_list = [EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,BusinessTravel,Department,EducationField,JobRole,MaritalStatus,JobInvolvement]

def calculate_weights_for_dummy_variables(dummy_variables_list): # dummy_variables_list è una lista di DataFrame, ognuno contenente colonne dummy per una variabile

    # Calcola la somma delle colonne dummy per ottenere il conteggio complessivo per ciascuna categoria
    total_counts = pd.concat(dummy_variables_list, axis=1).sum()

    # Calcola i pesi inversamente proporzionali alla frequenza di ciascuna categoria
    weights = pd.concat(dummy_variables_list, axis=1).shape[0] / total_counts

    # Normalizza i pesi in modo che la somma sia pari a 1
    weights /= weights.sum()

    # Crea un dizionario di pesi per le categorie
    category_weights = dict(zip(weights.index, weights))

    # Applica i pesi alle colonne dummy nei DataFrame originali
    weighted_dummy_variables_list = [dummy_vars.copy() for dummy_vars in dummy_variables_list]
    for i, dummy_vars in enumerate(weighted_dummy_variables_list):
        for col in dummy_vars.columns:
            weighted_dummy_variables_list[i][col] *= category_weights[col]

    return weighted_dummy_variables_list

weighted_dummy_variables_list = calculate_weights_for_dummy_variables(dummy_vars_list)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

# Creazione di un oggetto di regressione logistica
logreg = LogisticRegression()

# Definizione del selettore di feature RFE
selector = RFECV(logreg, step=1)

# Addestramento del selettore sul set di addestramento
selector.fit(Xe1, Ye1)

# Selezione delle variabili
selected_features = Xe1.columns[selector.support_]

# Addestramento del modello di regressione logistica sulle variabili selezionate
model_rfecv = sm.GLM(Ye1, Xe1[selected_features], family=sm.families.Binomial())
fit_rfecv = model_rfecv.fit()

# Stampa il riassunto del modello
print(fit_rfecv.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello con selezione delle variabili esplicative via metodo RFE per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1575.9

# Numero di parametri nel modello
num_parametri = 33+1

# Numero di osservazioni nel dataset di addestramento
n = len(Ye1)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Eseguo il test del rapporto di massima verosimiglianza tra modello ridotto con variabili significative al 5% e il modello con selezione delle variabili esplicative via metodo RFE-cv
from scipy.stats import chi2
# Calcolo il log-likelihood dei due modelli
ll_reduced = fit2.llf
ll_rfe = fit_rfecv.llf

# Calcola la statistica del test del rapporto di massima verosimiglianza
lr_statistic = -2 * (ll_rfe- ll_reduced)

# Gradi di libertà
df = fit2.df_model - fit_rfecv.df_model

# Calcola il p-value usando la distribuzione chi-quadro
p_value = 1.0 - chi2.cdf(lr_statistic, df)

# Stampare i risultati del test
print("Likelihood Ratio Test:")
print("Test statistic:", lr_statistic)
print("P-value:", p_value)
print("Degrees of freedom:", df)

# Confronto con un valore di soglia, ad esempio 0.05
alpha = 0.05
if p_value < alpha:
    print("Il modello ridotto con variabili significative al 5% è preferibile al modello con selezione delle variabili esplicative via metodo RFE-CV.")
else:
    print("Non ci sono prove sufficienti per rigettare il modello con selezione delle variabili esplicative via metodo RFE-cv.")

Dal criterio AICc è preferibile per poco il modello ridotto con variabili significative al 5%. Lo stesso risultato sembra che si ottenga anche dal test del rapporto di verosimiglianza, anche se con un valore di soglia dell'1% è preferibile il modello con selezione delle variabili esplicative via metodo RFE.

La scelta tra un valore di soglia del 5% o dell'1% dipende dalla tolleranza agli errori di tipo I e II e dalle norme del campo di studio. Un valore di soglia più basso rende il test più conservativo, ma può rendere più difficile rifiutare il null hypothesis.
In pratica, spesso si utilizza il valore di soglia del 5% come punto di partenza, ma in alcuni contesti più critici o in situazioni in cui è essenziale limitare gli errori di tipo I, si può scegliere un valore di soglia più basso come l'1%.

Se si è più conservativi e si vuole ridurre al minimo la probabilità di commettere un errore di tipo I (accettare erroneamente un falso effetto), si può essere più inclini a seguire la conclusione ottenuta con un valore di soglia dell'1%.

### Modelli Ridge e Lasso

Dato che si è in presenza di overfitting, l'uso di tecniche di regolarizzazione come Ridge e Lasso può essere utile per mitigare questo problema.

Si usa la cross-validation per selezionare il parametro di regolarizzazione ottimale (alfa) per Ridge e Lasso.
Questo aiuterà a trovare il giusto equilibrio tra adattamento ai dati di addestramento e generalizzazione ai dati di test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(Xe1):
    X_train, X_test = Xe1.iloc[train_index], Xe1.iloc[test_index]
    y_train, y_test = Ye1.iloc[train_index], Ye1.iloc[test_index]
    #y_train, y_test = Ye1[train_index], Ye1[test_index]#

#### Lasso

In [ ]:
# Regressione logistica con regolarizzazione L1 (LASSO)
lasso_model = LogisticRegression(penalty='l1', solver='liblinear')
lasso_model.fit(X_train, y_train)

# Predici sui dati di test
y_pred_lasso = lasso_model.predict(X_test)

# Valuta le performance
accuracy_lasso = accuracy_score(y_test, y_pred_lasso)
print("Accuracy con LASSO:", accuracy_lasso)

In [ ]:
# Calcola la log-verosimiglianza utilizzando il modulo statsmodels
log_likelihood = sm.Logit(y_train, sm.add_constant(X_train)).fit_regularized(method='l1').llf

# Calcola il criterio di Akaike corretto
num_params = len(lasso_model.coef_[0]) + 1  # Numero di parametri nel modello, inclusa l'intercetta
aicc = -2 * log_likelihood + 2 * num_params

print("Log-verosimiglianza di Lasso:", log_likelihood)
print("Criterio di Akaike corretto di Lasso:", aicc)

In [ ]:
num_coef = np.sum(lasso_model.coef_ != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_coef)

In [ ]:
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

# Calcola la matrice di confusione
conf_matrix_lasso = confusion_matrix(y_test, y_pred_lasso)
table = tabulate(conf_matrix_lasso, headers=['Predicted 0', 'Predicted 1'], showindex=['Actual 0', 'Actual 1'], tablefmt='grid')
# Stampa la matrice di confusione
print("Confusion Matrix (LASSO):")
print(table)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_lasso))

#### Ridge

In [ ]:
# Regressione logistica con regolarizzazione L2 (ridge)
ridge_model = LogisticRegression(penalty='l2', solver='liblinear')
ridge_model.fit(X_train, y_train)

# Predici sui dati di test
y_pred_ridge = ridge_model.predict(X_test)

# Valuta le performance
accuracy_ridge = accuracy_score(y_test, y_pred_ridge)
print("Accuracy con Ridge:", accuracy_ridge)

In [ ]:
# Estrai i coefficienti del modello
coefficients = ridge_model.coef_

# Calcola la log-verosimiglianza
log_likelihood = np.sum(y_train * np.log(ridge_model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - ridge_model.predict_proba(X_train)[:, 1]))

# Calcola l'AICc
n = len(y_train)  # Numero di osservazioni
k = len(coefficients.flatten())  # Numero di parametri nel modello

aic = -2 * log_likelihood + 2 * k  # AIC classico
aic_corrected = aic + (2 * k * (k + 1)) / (n - k - 1)  # AICc

print("Log-verosimiglianza di Ridge:", log_likelihood)
print("Criterio di Akaike corretto di Ridge::", aic_corrected)

In [ ]:
num_coef = np.sum(ridge_model.coef_ != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_coef)

In [ ]:
# Eseguo il test del rapporto di massima verosimiglianza tra modello Ridge e il modello Lasso
from scipy.stats import chi2
# Calcolo il log-likelihood dei due modelli
ll_lasso = sm.Logit(y_train, sm.add_constant(X_train)).fit_regularized(method='l1').llf
ll_ridge = np.sum(y_train * np.log(ridge_model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - ridge_model.predict_proba(X_train)[:, 1]))

# Calcola la statistica del test del rapporto di massima verosimiglianza
lr_statistic = -2 * (ll_ridge - ll_lasso)

# Gradi di libertà
df_lasso = num_params
df_ridge = k
df = df_lasso - df_ridge

# Calcola il p-value usando la distribuzione chi-quadro
p_value = 1.0 - chi2.cdf(lr_statistic, df)

# Stampare i risultati del test
print("Likelihood Ratio Test:")
print("Test statistic:", lr_statistic)
print("P-value:", p_value)
print("Degrees of freedom:", df)

# Confronto con un valore di soglia, ad esempio 0.05
alpha = 0.05
if p_value < alpha:
    print("Il modello Ridge è preferibile al modello Lasso.")
else:
    print("Non ci sono prove sufficienti per rigettare il modello Lasso.")

##### Ridge Regression

Ridge aggiunge una penalità quadratica alla somma dei quadrati dei coefficienti. Questo aiuta a "spalmare" i coefficienti e può essere efficace nel trattare il multicollinearity. Tuttavia, tiene comunque tutti i predittori nel modello.

ElasticNetCV è la classe in scikit-learn che implementa la regressione Elastic Net con validazione incrociata (cross-validation) per la selezione automatica del parametro di regolarizzazione. In questo caso, stiamo utilizzando Elastic Net, ma impostando l1_ratio=0.0, stiamo effettivamente selezionando solo il componente L2, che rende il modello equivalente a Ridge.

l1_ratio=0.0: Questo parametro controlla la proporzione tra le penalità L1 e L2 nella regressione Elastic Net. Quando l1_ratio è impostato a 1.0, significa che si sta utilizzando solo la penalità L1, rendendo il modello equivalente a Lasso. Se l1_ratio è impostato a 0.0, è equivalente a Ridge. Con 0.0 < l1_ratio < 1.0, si utilizzerebbe una combinazione di L1 e L2.

alphas=[0.1, 1.0, 10.0]: Questo è un elenco di valori di iperparametri alfa, che controllano la forza della regolarizzazione. ElasticNetCV eseguirà la ricerca del miglior valore di alfa durante la cross-validation. In questo caso, sta testando tre valori diversi: 0.1, 1.0 e 10.0.

cv=5: Questo parametro specifica la strategia di cross-validation. In questo caso, si sta utilizzando la cross-validation a 5 fold, il che significa che il set di dati viene diviso in 5 parti, e il modello viene addestrato e validato su ciascuna combinazione di 4 parti per l'addestramento e 1 parte per la validazione.

In [ ]:
from sklearn.linear_model import ElasticNetCV
# Inizializza il modello ElasticNetCV
Ridge = ElasticNetCV(l1_ratio=0.0, alphas=[0.1, 1.0, 10.0], cv=5)

In [ ]:
coefficients_ridge = []  # Memorizza i coefficienti ottenuti da ciascuna fold
for train_index, test_index in kf.split(Xe1):
    X_train, X_test = Xe1.iloc[train_index], Xe1.iloc[test_index]
    y_train, y_test = Ye1[train_index], Ye1[test_index]

    Ridge.fit(X_train, y_train)
    coefficients_ridge.append(Ridge.coef_)

In [ ]:
num_coef = np.sum(Ridge.coef_ != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_coef)

In [ ]:
# Calcola AICc e log-verosimiglianza utilizzando i coefficienti medi
coefficients_mean_r = np.mean(coefficients_ridge, axis=0)
n = len(Ye1)
mse = np.mean((Ye1 - np.dot(Xe1, coefficients_mean_r))**2)
df_model = np.sum(Ridge.coef_ != 0)  # Gradi di libertà del modello
log_likelihood = -n / 2 * (1 + np.log(2 * np.pi) + np.log(mse / n))
aicc = -2 * log_likelihood + 2 * df_model + 2 * df_model * (df_model + 1) / (n - df_model - 1)

# Stampa i risultati
print("Ridge - AICc:", aicc)
print("Log-likelihood:", log_likelihood)

##### Lasso Regression

Lasso può essere più utile se si ritiene che molte delle caratteristiche non siano rilevanti. Lasso aggiunge una penalità assoluta alla somma dei valori assoluti dei coefficienti, spingendo alcuni di essi a diventare esattamente zero.

In [ ]:
from sklearn.linear_model import ElasticNetCV
# Inizializza il modello ElasticNetCV
Lasso = ElasticNetCV(l1_ratio=1.0, alphas=[0.1, 1.0, 10.0], cv=5)

In [ ]:
coefficients = []  # Memorizza i coefficienti ottenuti da ciascuna fold
for train_index, test_index in kf.split(Xe1):
    X_train, X_test = Xe1.iloc[train_index], Xe1.iloc[test_index]
    y_train, y_test = Ye1[train_index], Ye1[test_index]

    Lasso.fit(X_train, y_train)
    coefficients.append(Lasso.coef_)

In [ ]:
# Calcola AICc e log-verosimiglianza utilizzando i coefficienti medi
coefficients_mean = np.mean(coefficients, axis=0)
n = len(Ye1)
mse = np.mean((Ye1 - np.dot(Xe1, coefficients_mean))**2)
df_model = np.sum(coefficients_mean != 0)  # Gradi di libertà del modello
log_likelihood = -n / 2 * (1 + np.log(2 * np.pi) + np.log(mse / n))
aicc = -2 * log_likelihood + 2 * df_model + 2 * df_model * (df_model + 1) / (n - df_model - 1)

# Stampa i risultati
print("Lasso - AICc:", aicc)
print("Log-likelihood:", log_likelihood)

In [ ]:
num_coef = np.sum(Lasso.coef_ != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_coef)

In [ ]:
from sklearn.model_selection import cross_val_predict
# Calcola le previsioni attraverso la cross-validation
y_pred = cross_val_predict(Lasso, Xe1, Ye1, cv=5)

##### Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNetCV
elasticnet_cv = ElasticNetCV(alphas=[0.1, 1.0, 10.0], l1_ratio=[0.25, 0.5, 0.75], cv=kf)
elasticnet_cv.fit(Xe1, Ye1)

## Divisione dati in set di addestramento e di test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Si seleziona le variabili esplicative
Xd = hr_logit_dummy.drop(['Attrition'], axis=1)

In [ ]:
# Si assegna a y la variabile risposta
Yd = hr_logit_dummy['Attrition']

In [ ]:
# Si dividono i dati in set di addestramento e di test (test del 30% e training del 70%)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(Xd, Yd, test_size=0.2, random_state=42)

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_d[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']] = scaler.fit_transform(X_train_d[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']])

X_test_d[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']] = scaler.fit_transform(X_test_d[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']])

La normalizzazione viene eseguita nel seguente modo:

- Crea un'istanza della classe StandardScaler assegnandola alla variabile scaler.
- Applica la normalizzazione alle colonne specificate del dataframe X_train utilizzando il metodo fit_transform() della classe StandardScaler.
- Il risultato della normalizzazione viene assegnato di nuovo alle stesse colonne nel dataframe X_train.

La normalizzazione delle features rende i dati comparabili tra loro. Utilizzando lo StandardScaler, le colonne specificate vengono trasformate in modo tale che abbiano una media pari a 0 e una deviazione standard pari a 1. Questo aiuta a evitare che le variabili con valori più grandi abbiano un'influenza sproporzionata sui modelli.

In [ ]:
# Check sul tasso di abbandono
attrition_rate = (sum(hr_logit_dummy['Attrition']) / len(hr_logit_dummy['Attrition'].index)) * 100
print("Il tasso di abbandono rispetto all'anno precedente è del {:.2f}%".format(attrition_rate))

## Costruzione del modello completo con variabili dummy

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
X_train_d = sm.add_constant(X_train_d)
X_test_d = sm.add_constant(X_test_d)

In [ ]:
import statsmodels.api as sm

# Stima del modello completo di regressione logistica
log_model = sm.GLM(y_train_d, X_train_d, family=sm.families.Binomial())
fit0 = log_model.fit()

# Stampa il riassunto del modello
print(fit0.summary())

Nell'output del modello di regressione logistica, si hanno diverse informazioni statistiche utili per valutare la bontà del modello. Ecco alcune considerazioni basate sull'output:

1. Coefficienti: I coefficienti stimati per le variabili indipendenti del modello sono elencati nella tabella dei risultati. Ogni coefficiente rappresenta la variazione media nell'odds ratio dell'evento di Attrition per un aumento unitario nella variabile indipendente corrispondente. Ad esempio, il coefficiente stimato per la variabile "MaritalStatus_Married" è 0.1850. Ciò significa che un aumento di una unità in "MaritalStatus_Married" è associato ad un aumento dell'odds ratio dell'Attrition di circa il 18,50%.

2. Valori p: I valori p indicano la significatività statistica dei coefficienti stimati. Nell'output, il valore p è elencato nella colonna "P>|z|". Ad esempio, per la variabile "Nr Ore", il valore p è 0.663. Un valore p elevato indica che non c'è evidenza sufficiente per respingere l'ipotesi nulla di non significatività della variabile. Pertanto, si potrebbe pensare di escludere la variabile "Nr Ore".

3. Pseudo R-quadrato: il Pseudo R-squared di Cragg & Uhler (CS) è una misura della bontà di adattamento del modello. In questo modello il Pseudo R-quadrato (CS) è 0.1713. Questo valore indica che il modello spiega circa il 17% della variazione nell'Attrition.

4. Devianza e Pearson chi2: Questi sono test di bontà di adattamento per il modello. Valori più alti di devianza e Pearson chi2 indicano una peggiore aderenza del modello ai dati. In questo caso, la devianza è 2490.4 e il Pearson chi2 è 4.27e+03. Questi valori sono molto elevati rispetto al numero di gradi di libertà del modello (53).

5. Intervallo di confidenza: Nella colonna "[0.025 0.975]" sono forniti gli estremi degli intervalli di confidenza al 95% per i coefficienti stimati. Gli intervalli di confidenza possono aiutare a valutare la precisione delle stime dei coefficienti.

### Valutazione del modello completo

In [ ]:
y_pred_d = fit0.predict(X_test_d)

# Valutazione delle previsioni
y_pred_d_binary = (y_pred_d > 0.5).astype(int)  # Converte le previsioni in valori binari
accuracy_d = (y_test_d == y_pred_d_binary).mean()
print("Accuracy:", accuracy_d)

In [ ]:
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
# Calcolo della matrice di confusione
cmd = confusion_matrix(y_test_d, y_pred_d_binary)
table_d = tabulate(cmd, headers=['Predicted 0', 'Predicted 1'], showindex=['Actual 0', 'Actual 1'], tablefmt='grid')
print("Confusion Matrix:")
print(table_d)

L'output sopra riportato rappresenta la matrice di confusione, che fornisce una panoramica delle previsioni effettuate dal modello completo rispetto alle etichette di classe reali. La matrice di confusione è organizzata in una tabella che mostra il numero di casi per ogni combinazione di previsioni e valori reali delle classi.

- La colonna "Predicted 1" rappresenta le previsioni per la classe 1 (logoramento) mentre la colonna "Predicted 0" rappresenta le previsioni effettuate dal modello per la classe 0 (non logoramento).
- La riga "Actual 1" rappresenta i valori reali delle osservazioni appartenenti alla classe 1 mentre la riga "Actual 0" rappresenta i valori reali delle osservazioni appartenenti alla classe 0.
- I numeri all'interno della tabella rappresentano il conteggio dei casi per ogni combinazione di previsioni e valori reali delle classi. Ad esempio, ci sono 28 casi in cui sia la previsione che il valore reale corrispondono alla classe 1, ci sono 730 casi in cui sia la previsione che il valore reale corrispondono alla classe 0, ecc.

I casi correttamente classificati sono sulla diagonale principale della matrice. Ad esempio, i 730 casi correttamente classificati come "Predicted 0" e "Actual 0" rappresentano i veri negativi.
Gli elementi al di fuori della diagonale principale rappresentano gli errori di classificazione. Ad esempio, i 103 casi classificati erroneamente come "Predicted 0" ma in realtà appartenenti alla classe 1 rappresentano i falsi negativi, mentre i 21 casi classificati erroneamente come "Predicted 1" ma in realtà appartenenti alla classe 0 rappresentano i falsi positivi.

La matrice di confusione fornisce un'indicazione della capacità del modello di distinguere correttamente tra le classi. Si possono utilizzare i numeri nella matrice per calcolare metriche di valutazione come la precisione, il richiamo e l'F1-score, che offrono una valutazione più completa delle prestazioni del modello.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_d,y_pred_d_binary))

1. Precision (Precisione):

    - La precisione per la classe 0 (etichetta negativa) è 0.88 [730/(730+103)], che indica la percentuale di casi classificati correttamente come negativi rispetto al totale dei casi classificati come negativi (colonna Predicted 0 della matrice di confusione).
    - La precisione per la classe 1 (etichetta positiva) è 0.57 [28/(21+28)], che indica la percentuale di casi classificati correttamente come positivi rispetto al totale dei casi classificati come positivi (colonna Predicted 1 della matrice di confusione).
    
    Un valore più alto di Precision indica una maggiore accuratezza nelle previsioni per la classe specifica.

2. Recall (Richiamo):

    - Il richiamo o sensitività (sensitivity) per la classe 0 è 0.97 [730/(730+21], che rappresenta la percentuale di casi negativi correttamente identificati rispetto al totale dei casi reali negativi (riga Actual 0 della matrice di confusione).
    - Il richiamo o sensitività (sensitivity) per la classe 1 è 0.21 [28/(28+103)], che rappresenta la percentuale di casi positivi correttamente identificati rispetto al totale dei casi reali positivi (riga Actual 1 della matrice di confusione).
    
    Un valore più alto di Recall (varia da 0 a 1) indica una migliore capacità del modello di identificare correttamente i casi della classe specifica.

3. F1-score:

    - L'F1-score per la classe 0 è 0.92 [2*(0.88*0.97)/(0.88+0.97)], che rappresenta una misura complessiva dell'accuratezza del modello considerando sia la precisione che il richiamo.
    - L'F1-score per la classe 1 è 0.31 [2*(0.57*0.21)/(0.57+0.21)], che rappresenta una misura complessiva dell'accuratezza del modello per la classe positiva.
    
    L'F1-score è la media armonica della precisione e del richiamo [2(Precision*Recall)/(Precision+Recall)] ed è utile per valutare il bilanciamento tra precisione e richiamo.
    un F1-score di 0.31 (dove il valore può variare da 0 a 1) per la classe 1 indica che il modello ha una bassa precisione e un basso richiamo per quella classe. Ciò suggerisce che il modello fatica a identificare correttamente la maggior parte delle istanze positive (1) e tende a fare molte previsioni errate, sia falsi positivi che falsi negativi, per questa classe.
    Sussiste una disparità significativa tra le due classi, con prestazioni migliori per la classe negativa (0) rispetto alla classe positiva (1).
    
4. Support:

    - Il supporto per la classe 0 è 751, che rappresenta il numero totale di casi appartenenti alla classe negativa.
    - Il supporto per la classe 1 è 131, che rappresenta il numero totale di casi appartenenti alla classe positiva.
    
    Il supporto indica la distribuzione dei casi nelle classi di destinazione.

5. Accuracy (Accuratezza):

    - L'accuratezza complessiva del modello è 0.86, che rappresenta la percentuale di casi classificati correttamente (vero positivo + vero negativo) rispetto al totale dei casi.

    L'accuratezza fornisce una misura complessiva delle prestazioni del modello.
    un'accuratezza del 0.86 suggerisce che il modello ha una prestazione complessiva abbastanza buona, in quanto classifica correttamente l'86% delle istanze. Tuttavia, è importante considerare il contesto del problema e il bilanciamento delle classi nel set di dati.
    Le classi nel set di dati sono sbilanciate: l'accuratezza non è la migliore metrica per valutare le prestazioni del modello in quanto si ha una classe di minoranza molto più piccola rispetto alla classe di maggioranza.

6. Macro avg e weighted avg:

    - Macro avg rappresenta la media non ponderata delle metriche per tutte le classi.
        - Precision: 0.72 indica una media delle precisioni per tutte le classi del 72%. Questo valore indica un buon equilibrio generale tra precisione per le diverse classi.
        -   Recall: 0.59 indica una media dei recall per tutte le classi del 59%. Questo valore suggerisce che, in media, il modello sta classificando correttamente solo il 59% delle istanze positive per tutte le classi.
        -   F1-score: 0.62 indica una media degli F1-score per tutte le classi del 62%. Questo valore riflette un bilanciamento tra precisione e recall mediamente buono per tutte le classi.

    - Weighted avg rappresenta la media ponderata delle metriche per tutte le classi, ponderate in base al supporto di ciascuna classe.
        -   Precision: 0.83 indica una media ponderata delle precisioni per tutte le classi dell'83%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette una precisione complessivamente buona, considerando il peso delle diverse classi.
        -   Recall: 0.86 indica una media ponderata dei recall per tutte le classi dell'86%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette un buon recall complessivo, considerando il peso delle diverse classi.
        -   F1-score: 0.83 indica una media ponderata degli F1-score per tutte le classi dell'80%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette un bilanciamento complessivamente buono tra precisione e recall, considerando il peso delle diverse classi.

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1245.2

# Numero di parametri nel modello
num_parametri = 53+1

# Numero di osservazioni nel dataset di addestramento
n = len(y_train_d)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Si calcola il BIC (Bayesian Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

 # Log-likelihood del modello
log_likelihood = fit0.llf

# Numero di parametri del modello (è uguale al numero di coefficienti nel modello + 1 per l'intercetta)
num_parametri = fit0.df_model + 1

# Numero di osservazioni nel set di dati
n = len(y_train_d)

# Calcolo il BIC
BIC = -2 * log_likelihood + num_parametri * np.log(n)

# Un valore BIC inferiore indica una migliore adeguabilità del modello ai dati, tenendo conto della complessità del modello.

BIC

### Tecniche di regolarizzazione: Regressione di Ridge & Regressione di Lasso

Dato che il modello completo ha un basso valore di bontà d'adattamento (vedi il log-likelihood molto basso) e una devianza elevata, potrebbe indicare che il modello è sovradattato (overfitting) ai dati di addestramento. Ciò significa che il modello si adatta troppo bene ai dati di addestramento, ma non generalizza bene ai nuovi dati.
In questo caso si considera di utilizzare tecniche di regolarizzazione.

La regressione Ridge e la regressione Lasso sono modelli lineari che aiutano a prevenire l'overfitting ed a gestire il problema della multicolinarietà (correlazione tra le variabili predittive). Ridge e LASSO introducono un termine di penalizzazione nella funzione di regressione per limitare i coefficienti delle variabili. Ridge tende a ridurre tutti i coefficienti senza annullarli, mentre LASSO può annullare alcuni coefficienti, portando ad una selezione automatica delle variabili.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

#### Regressione Ridge

In [ ]:
# Intervallo di alpha: Si esegue una ricerca dell'intervallo di alpha utilizzando la ricerca griglia adattiva.
ridge_cv = RidgeCV(cv=5)
ridge_cv.fit(X_train_d, y_train_d)

nella classe RidgeCV, non è necessario specificare il numero di valori di alpha (n_alphas) da testare né è necessario specificare la tolleranza di convergenza come si farà nella regressione Lasso. La classe RidgeCV esegue automaticamente la ricerca griglia adattiva per determinare il miglior valore di alpha utilizzando la validazione incrociata.

In [ ]:
# Ottieni il miglior valore di alpha selezionato dalla ricerca griglia adattiva
best_alpha_ridge = ridge_cv.alpha_
print("Best alpha (Ridge CV):", best_alpha_ridge)

La regressione Ridge utilizza il parametro alpha come termine di regolarizzazione per controllare il trade-off tra la riduzione del modello (per ridurre il rischio di overfitting) e l'adeguamento ai dati di addestramento. Un valore di alpha più alto corrisponde a una regolarizzazione più forte, che penalizza i coefficienti di regressione più grandi e riduce la complessità del modello.

In questo caso il valore di di alpha è pari a 10.0, questo implica che una regolarizzazione più forte è preferibile per il modello Ridge.

##### Modello di regressione logistica regolarizzata secondo Ridge

In [ ]:
fit_ridge = log_model.fit_regularized(method='elastic_net', alpha=1/10, L1_wt=0)

In [ ]:
y_pred_ridge = fit_ridge.predict(X_test_d)
ridge_mse = mean_squared_error(y_test_d, y_pred_ridge)
print("MSE (Ridge):", ridge_mse)

un MSE più vicino a zero indica una migliore bontà di adattamento del modello ai dati di test.

Poiché il modello Ridge è una regressione lineare regolarizzata e non viene fornita una stima diretta del log-likelihood, per calcolare l'AIC si può usare il punteggio di informazione di Akaike modificato (AICc), che tiene conto delle dimensioni del campione e del numero di parametri. L'AICc è una versione corretta dell'AIC per campioni di dimensioni relativamente piccole.

In [ ]:
# Calcolo del numero di parametri
num_parameters_ridge = np.count_nonzero(fit_ridge.params)
# Calcolo del numero di osservazioni
n = len(y_test_d)
# Calcolo dell'AICc
aic_ridge = n * np.log(ridge_mse) + 2 * num_parameters_ridge + (2 * num_parameters_ridge * (num_parameters_ridge + 1)) / (n - num_parameters_ridge - 1)
print("AICc (Ridge):", aic_ridge)

In [ ]:
num_coef = sum(fit_ridge.params != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_coef)

#### Regressione Lasso

In [ ]:
# Intervallo di alpha: Si esegue una ricerca dell'intervallo di alpha utilizzando la ricerca griglia adattiva.
lasso_cv = LassoCV(cv=5, n_alphas=100, tol=0.001)
lasso_cv.fit(X_train_d, y_train_d)

In [ ]:
# Ottenere il miglior valore di alpha
best_alpha_lasso = lasso_cv.alpha_
print("Best alpha (Lasso CV):", best_alpha_lasso)

La classe LassoCV esegue automaticamente la ricerca griglia adattiva, provando diversi valori di alpha su una scala logaritmica. Utilizzando la validazione incrociata, stima le prestazioni del modello per ogni valore di alpha e seleziona quello che fornisce le migliori prestazioni medie.

Poiché si ottiene un valore di alpha prossimo allo zero, le stime LASSO dei coefficienti tendono a essere molto simili alle stime classiche dei minimi quadrati; comunque potrebbero esserci piccole differenze dovute alla presenza della regolarizzazione.

##### Modello di regressione logistica regolarizzata secondo Lasso

In [ ]:
fit_lasso = log_model.fit_regularized(method='elastic_net', alpha=0.0004509763925403281, L1_wt=1)

In [ ]:
y_pred_lasso = fit_lasso.predict(X_test_d)
lasso_mse = mean_squared_error(y_test_d, y_pred_lasso)
print("MSE (Lasso):", lasso_mse)

un MSE più vicino a zero indica una migliore bontà di adattamento del modello ai dati di test.

In [ ]:
# Calcolo del numero di parametri
num_parameters_lasso = np.count_nonzero(fit_lasso.params)
# Calcolo del numero di osservazioni
n = len(y_test_d)
# Calcolo dell'AICc
aic_lasso = n * np.log(lasso_mse) + 2 * num_parameters_lasso + (2 * num_parameters_lasso * (num_parameters_lasso + 1)) / (n - num_parameters_lasso - 1)
print("AICc (Lasso):", aic_lasso)

In [ ]:
num_zero_coef = sum(fit_lasso.params == 0)
print("Numero di variabili esplicative con coefficienti pari a zero:", num_zero_coef)

In [ ]:
num_no_zero_coef = sum(fit_lasso.params != 0)
print("Numero di variabili esplicative con coefficienti diversi da zero:", num_no_zero_coef)

## Costruzione del modello considerando le variabili risultanti significative dall'EDA

Variabili quantitative considerate rilevanti dall'analisi esplorativa attraverso il T-test sono:

	Variabili con differenza significativa tra i gruppi di Attrition sia con alpha = 0.05 che con alpha = 0.01:
		1. Age
		2. NumCompaniesWorked
		3. TotalWorkingYears
		4. TrainingTimesLastYear
		5. YearsAtCompany
		6. YearsWithCurrManager

	Variabili con differenza significativa tra i gruppi di Attrition solo per alpha = 0.05:
		1. MonthlyIncome_Euro
		2. PercentSalaryHike
		3. YearsSinceLastPromotion

Variabili categoriali considerate rilevanti dall'analisi esplorativa attraverso il test del Chi-Quadro sono:

	Variabili con differenza significativa tra i gruppi di Attrition sia con alpha = 0.05 che con alpha = 0.01:
		1. EnvironmentSatisfaction
		2. JobSatisfaction
		3. WorkLifeBalance
		4. BusinessTravel
		5. Department
		6. EducationField
		7. JobRole
		8. MaritalStatus

	Variabili con differenza significativa tra i gruppi di Attrition solo per alpha = 0.05:
		1. JobInvolvement

In [ ]:
hr_eda = hr_logit_dummy[['Attrition','Age','NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear',
            'YearsAtCompany','YearsWithCurrManager','EnvironmentSatisfaction_Low',
            'EnvironmentSatisfaction_Medium','EnvironmentSatisfaction_Very High','JobSatisfaction_Low',
            'JobSatisfaction_Medium','JobSatisfaction_Very High','WorkLifeBalance_Best',
            'WorkLifeBalance_Better','WorkLifeBalance_Good','BusinessTravel_Travel_Frequently',
            'BusinessTravel_Travel_Rarely','Department_Research & Development','Department_Sales',
            'EducationField_Life Sciences','EducationField_Marketing','EducationField_Medical','EducationField_Other',
            'EducationField_Technical Degree','JobRole_Human Resources','JobRole_Laboratory Technician','JobRole_Manager',
            'JobRole_Manufacturing Director','JobRole_Manufacturing Director','JobRole_Research Director','JobRole_Research Scientist',
            'JobRole_Sales Executive','JobRole_Sales Representative','MaritalStatus_Married','MaritalStatus_Single']]

In [ ]:
# Si codifica la variabile risposta in 0 e 1
hr_eda['Attrition'] = hr_eda['Attrition'].replace({'Yes': 1, "No": 0})

In [ ]:
Ye = hr_eda['Attrition']
Xe = hr_eda.drop('Attrition',axis=1)

In [ ]:
X_train_e,X_test_e, y_train_e, y_test_e = train_test_split(Xe,Ye, test_size = 0.20, random_state=42)

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
scaler = StandardScaler()

X_train_e[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsWithCurrManager','Age']] = scaler.fit_transform(X_train_e[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
            'YearsWithCurrManager','Age']])

X_test_e[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsWithCurrManager','Age']] = scaler.fit_transform(X_test_e[['NumCompaniesWorked','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
            'YearsWithCurrManager','Age']])

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
X_train_e = sm.add_constant(X_train_e)
X_test_e = sm.add_constant(X_test_e)

In [ ]:
# Stima del modello completo di regressione logistica
log_model_e = sm.GLM(y_train_e, X_train_e, family=sm.families.Binomial())
fit1 = log_model.fit()

# Stampa il riassunto del modello
print(fit1.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1281.9

# Numero di parametri nel modello
num_parametri = 34+1

# Numero di osservazioni nel dataset di addestramento
n = len(y_train_e)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AICc

### Regressione Ridge

In [ ]:
# Intervallo di alpha: Si esegue una ricerca dell'intervallo di alpha utilizzando la ricerca griglia adattiva.
e_ridge_cv = RidgeCV(cv=5)
e_ridge_cv.fit(X_train_e, y_train_e)

nella classe RidgeCV, non è necessario specificare il numero di valori di alpha (n_alphas) da testare né è necessario specificare la tolleranza di convergenza come si farà nella regressione Lasso. La classe RidgeCV esegue automaticamente la ricerca griglia adattiva per determinare il miglior valore di alpha utilizzando la validazione incrociata.

In [ ]:
# Ottieni il miglior valore di alpha selezionato dalla ricerca griglia adattiva
e_best_alpha_ridge = e_ridge_cv.alpha_
print("Best alpha (Ridge CV of EDA Model):", e_best_alpha_ridge)

#### Modello di regressione logistica regolarizzata secondo Ridge per modello parziale

In [ ]:
fit_ridge_e = log_model_e.fit_regularized(method='elastic_net', alpha=1/10, L1_wt=0)

In [ ]:
y_pred_ridge_e = fit_ridge_e.predict(X_test_e)
ridge_e_mse = mean_squared_error(y_test_e, y_pred_ridge_e)
print("MSE (Ridge of EDA Model):", ridge_mse)

In [ ]:
# Calcolo del numero di parametri
num_parameters_ridge_e = np.count_nonzero(fit_ridge_e.params)
# Calcolo del numero di osservazioni
n = len(y_test_e)
# Calcolo dell'AICc
aic_ridge_e = n * np.log(ridge_e_mse) + 2 * num_parameters_ridge_e + (2 * num_parameters_ridge_e * (num_parameters_ridge_e + 1)) / (n - num_parameters_ridge_e - 1)
print("AICc (Ridge of EDA Model):", aic_ridge_e)

### Regressione Lasso

In [ ]:
# Intervallo di alpha: Si esegue una ricerca dell'intervallo di alpha utilizzando la ricerca griglia adattiva.
e_lasso_cv = LassoCV(cv=5, n_alphas=100, tol=0.001)
e_lasso_cv.fit(X_train_e, y_train_e)

In [ ]:
# Ottenere il miglior valore di alpha
e_best_alpha_lasso = e_lasso_cv.alpha_
print("Best alpha (Lasso CV of EDA Model):", e_best_alpha_lasso)

#### Modello di regressione logistica regolarizzata secondo Lasso per modello parziale

In [ ]:
fit_lasso_e = log_model_e.fit_regularized(method='elastic_net', alpha=6.392473845158518e-05, L1_wt=1)

In [ ]:
y_pred_lasso_e = fit_lasso_e.predict(X_test_e)
lasso_e_mse = mean_squared_error(y_test_e, y_pred_lasso_e)
print("MSE (Lasso of EDA Model):", lasso_e_mse)

In [ ]:
# Calcolo del numero di parametri
num_parameters_lasso_e = np.count_nonzero(fit_lasso_e.params)
# Calcolo del numero di osservazioni
n = len(y_test_e)
# Calcolo dell'AICc
aic_lasso_e = n * np.log(lasso_e_mse) + 2 * num_parameters_lasso_e + (2 * num_parameters_lasso_e * (num_parameters_lasso_e + 1)) / (n - num_parameters_lasso_e - 1)
print("AICc (Lasso of EDA Model):", aic_lasso_e)

In [ ]:
num_zero_coef_e = sum(fit_lasso_e.params == 0)
print("Numero di variabili esplicative con coefficienti pari a zero:", num_zero_coef_e)

## Eliminazione di alcune variabili (anche dummy)

### Si visualizza la matrice di correlazione tra tutte le variabili
plt.figure(figsize = (50,40))   
sns.heatmap(hr_logit.corr(),annot = True,cmap="tab20c")
plt.show()

### Si genera un grafico a barre delle correlazioni tra la variabile risposta "Attrition" e le altre variabili nel dataset hr_logit, ordinando le variabili in base alla correlazione in modo decrescente. Questo aiuta a identificare le variabili che potrebbero avere una maggiore influenza sull'attrition dei dipendenti
plt.figure(figsize=(20,8))
hr_logit.corr()['Attrition'].sort_values(ascending = False).plot(kind='bar')

Dall'analisi esplorativa dei dati si è riscontrato come le variabili quantitative Age, MonthlyIncome, NumCompaniesWorked, PercentSalaryHike, TotalWorkingYears, TrainingTimesLastYear, YearsAtCompany, YearsSinceLastPromotion, YearsWithCurrManager potessero essere statisticamente significative con alpha = 0.05.

Dall'analisi esplorativa dei dati si è riscontrato come le variabili categoriali JobInvolvement, EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance, BusinessTravel, Department, EducationField, JobRole, MaritalStatus potessero essere statisticamente significative con alpha = 0.05.

Il grafico a barre sopra riportato ci mostra un quadro complessivo, sia per variabili categoriali che quantitative.

### Si gestiscono le variabili dummy altamente correlate nel dataframe di addestramento

corrmat = X_train.corr() # Si calcola la matrice di correlazione tra tutte le variabili nel dataframe X_train
corrdf = corrmat.where(np.triu(np.ones(corrmat.shape), k=1).astype(bool)) ### Si crea un nuovo dataframe corrdf in cui vengono mantenute solo le
### correlazioni sopra la diagonale principale della matrice di correlazione. Questo evita la duplicazione delle informazioni e riduce la matrice a una forma triangolare superiore.
corrdf = corrdf.unstack().reset_index() ### Si trasforma la matrice di correlazione in un formato tabellare con tre colonne:
### 'Var1', 'Var2' e 'Correlation'. Ogni riga rappresenta una coppia di variabili e la loro correlazione.
corrdf.columns = ['Var1', 'Var2', 'Correlation'] ### Si rinomina le colonne del dataframe corrdf in 'Var1', 'Var2' e 'Correlation'
### per una migliore leggibilità.
corrdf.dropna(subset = ['Correlation'], inplace = True) ### Si rimuovono le righe che contengono valori mancanti nella colonna 'Correlation'.
corrdf['Correlation'] = round(corrdf['Correlation'], 2) ### Si arrotondano i valori di correlazione a due decimali.
corrdf['Correlation'] = abs(corrdf['Correlation']) ### Si calcola il valore assoluto delle correlazioni.
### In questo modo, i valori positivi e negativi della correlazione vengono trattati allo stesso modo per identificare le variabili altamente correlate.
matrix= corrdf.sort_values(by = 'Correlation', ascending = False).head(50) ### Si crea un nuovo dataframe chiamato matrix che contiene le prime 50 righe
### del dataframe corrdf, ordinate in base alla correlazione in ordine decrescente.

matrix ### matrix rappresenta le coppie di variabili con le correlazioni più alte.

Il codice esegue una serie di operazioni per identificare le variabili dummy altamente correlate nel dataframe X_train. Vengono calcolate le correlazioni tra le variabili, quindi viene creata una tabella delle correlazioni ordinata per valori assoluti. Infine, viene selezionato un subset di coppie di variabili con le correlazioni più alte (le prime 50) nel dataframe matrix. Questo è utile per individuare e gestire la multicollinearità tra le variabili


### Si memorizza in un elenco gli elementi di 'Var2' (rimuovendo eventuali duplicati attraverso 'set()')
unique=list(set(matrix.Var2))
print("Il numero di elementi unici presenti nell\'elenco è di",len(unique))

### Si rimuovono dal dataframe X_test e X_train le colonne indicate dall'elenco unique. La funzione drop viene utilizzata per rimuovere le colonne in base alle etichette (il parametro columns si usa per indicare le etichette delle colonne che si desidera eliminare)
X_test = X_test.drop(columns=unique)
X_train = X_train.drop(columns=unique)

## Costruzione del modello senza variabili dummy

In [ ]:
hr_log = hr[['Nr Ore','JobInvolvement','PerformanceRating','EnvironmentSatisfaction','JobSatisfaction','WorkLifeBalance','Age',
'Attrition','DistanceFromHome','Education','JobLevel','MonthlyIncome_Euro','NumCompaniesWorked','PercentSalaryHike','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany',
'YearsSinceLastPromotion','YearsWithCurrManager']]

In [ ]:
# Si codifica la variabile risposta in 0 e 1
hr_log['Attrition'] = hr_log['Attrition'].replace({'Yes': 1, "No": 0})

In [ ]:
y = hr_log['Attrition']
x = hr_log.drop('Attrition',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

In [ ]:
import statsmodels.api as sm

# Aggiungi una colonna di costante a X_train
X_train = sm.add_constant(X_train)

# Crea un modello di regressione logistica
glm_model = sm.GLM(y_train, X_train, family=sm.families.Binomial())

# Addestramento del modello
glm_results = glm_model.fit()

# Previsioni sui dati di test
X_test = sm.add_constant(X_test)
y_pred = glm_results.predict(X_test)

# Valutazione delle previsioni
y_pred_binary = (y_pred > 0.5).astype(int)  # Converte le previsioni in valori binari
accuracy = (y_test == y_pred_binary).mean()
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
# Calcolo della matrice di confusione
cm = confusion_matrix(y_test, y_pred_binary)
table = tabulate(cm, headers=['Predicted 0', 'Predicted 1'], showindex=['Actual 0', 'Actual 1'], tablefmt='grid')
print("Confusion Matrix:")
print(table)

L'output sopra riportato rappresenta la matrice di confusione, che fornisce una panoramica delle previsioni effettuate dal modello rispetto alle etichette di classe reali. La matrice di confusione è organizzata in una tabella che mostra il numero di casi per ogni combinazione di previsioni e valori reali delle classi.

- La colonna "Predicted 1" rappresenta le previsioni per la classe 1 (logoramento) mentre la colonna "Predicted 0" rappresenta le previsioni effettuate dal modello per la classe 0 (non logoramento).
- La riga "Actual 1" rappresenta i valori reali delle osservazioni appartenenti alla classe 1 mentre la riga "Actual 0" rappresenta i valori reali delle osservazioni appartenenti alla classe 0.
- I numeri all'interno della tabella rappresentano il conteggio dei casi per ogni combinazione di previsioni e valori reali delle classi. Ad esempio, ci sono 747 casi in cui sia la previsione che il valore reale corrispondono alla classe 1, ci sono 8 casi in cui sia la previsione che il valore reale corrispondono alla classe 0, ecc.

I casi correttamente classificati sono sulla diagonale principale della matrice. Ad esempio, i 747 casi correttamente classificati come "Predicted 1" e "Actual 01" rappresentano i veri positivi.
Gli elementi al di fuori della diagonale principale rappresentano gli errori di classificazione. Ad esempio, i 4 casi classificati erroneamente come "Predicted 0" ma in realtà appartenenti alla classe 1 rappresentano i falsi negativi, mentre i 123 casi classificati erroneamente come "Predicted 1" ma in realtà appartenenti alla classe 0 rappresentano i falsi positivi.

La matrice di confusione fornisce un'indicazione della capacità del modello di distinguere correttamente tra le classi. Si possono utilizzare i numeri nella matrice per calcolare metriche di valutazione come la precisione, il richiamo e l'F1-score, che offrono una valutazione più completa delle prestazioni del modello.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_binary))

1. Precision (Precisione):

    - La precisione per la classe 0 (etichetta negativa) è 0.86 [747/(747+123)], che indica la percentuale di casi classificati correttamente come negativi rispetto al totale dei casi classificati come negativi (colonna Predicted 0 della matrice di confusione).
    - La precisione per la classe 1 (etichetta positiva) è 0.67 [8/(8+4)], che indica la percentuale di casi classificati correttamente come positivi rispetto al totale dei casi classificati come positivi (colonna Predicted 1 della matrice di confusione).
    
    Un valore più alto di Precision indica una maggiore accuratezza nelle previsioni per la classe specifica.

2. Recall (Richiamo):

    - Il richiamo o sensitività (sensitivity) per la classe 0 è 0.99 [747/(747+4)], che rappresenta la percentuale di casi negativi correttamente identificati rispetto al totale dei casi reali negativi (riga Actual 0 della matrice di confusione).
    - Il richiamo o sensitività (sensitivity) per la classe 1 è 0.06 [8/(8+123)], che rappresenta la percentuale di casi positivi correttamente identificati rispetto al totale dei casi reali positivi (riga Actual 1 della matrice di confusione).
    
    Un valore più alto di Recall (varia da 0 a 1) indica una migliore capacità del modello di identificare correttamente i casi della classe specifica.

3. F1-score:

    - L'F1-score per la classe 0 è 0.92 [2*(0.86*0.99)/(0.86+0.99)], che rappresenta una misura complessiva dell'accuratezza del modello considerando sia la precisione che il richiamo.
    - L'F1-score per la classe 1 è 0.11 [2*(0.67*0.06)/(0.67+0.06)], che rappresenta una misura complessiva dell'accuratezza del modello per la classe positiva.
    
    L'F1-score è la media armonica della precisione e del richiamo [2(Precision*Recall)/(Precision+Recall)] ed è utile per valutare il bilanciamento tra precisione e richiamo.
    un F1-score di 0.11 per la classe 1 indica che il modello ha una bassa precisione e un basso richiamo per quella classe. Ciò suggerisce che il modello fatica a identificare correttamente la maggior parte delle istanze positive (1) e tende a fare molte previsioni errate, sia falsi positivi che falsi negativi, per questa classe.
    Sussiste una disparità significativa tra le due classi, con prestazioni migliori per la classe negativa (0) rispetto alla classe positiva (1).
    
4. Support:

    - Il supporto per la classe 0 è 751, che rappresenta il numero totale di casi appartenenti alla classe negativa.
    - Il supporto per la classe 1 è 131, che rappresenta il numero totale di casi appartenenti alla classe positiva.
    
    Il supporto indica la distribuzione dei casi nelle classi di destinazione.

5. Accuracy (Accuratezza):

    - L'accuratezza complessiva del modello è 0.86, che rappresenta la percentuale di casi classificati correttamente (vero positivo + vero negativo) rispetto al totale dei casi.

    L'accuratezza fornisce una misura complessiva delle prestazioni del modello.
    un'accuratezza del 0.86 suggerisce che il modello ha una prestazione complessiva abbastanza buona, in quanto classifica correttamente l'86% delle istanze. Tuttavia, è importante considerare il contesto del problema e il bilanciamento delle classi nel set di dati.
    Le classi nel set di dati sono sbilanciate: l'accuratezza non è la migliore metrica per valutare le prestazioni del modello in quanto si ha una classe di minoranza molto più piccola rispetto alla classe di maggioranza.

6. Macro avg e weighted avg:

    - Macro avg rappresenta la media non ponderata delle metriche per tutte le classi.
        - Precision: 0.76 indica una media delle precisioni per tutte le classi del 76%. Questo valore indica un buon equilibrio generale tra precisione per le diverse classi.
        -   Recall: 0.53 indica una media dei recall per tutte le classi del 53%. Questo valore suggerisce che, in media, il modello sta classificando correttamente solo il 53% delle istanze positive per tutte le classi.
        -   F1-score: 0.52 indica una media degli F1-score per tutte le classi del 52%. Questo valore riflette un bilanciamento tra precisione e recall mediamente basso per tutte le classi.

    - Weighted avg rappresenta la media ponderata delle metriche per tutte le classi, ponderate in base al supporto di ciascuna classe.
        -   Precision: 0.83 indica una media ponderata delle precisioni per tutte le classi dell'83%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette una precisione complessivamente buona, considerando il peso delle diverse classi.
        -   Recall: 0.86 indica una media ponderata dei recall per tutte le classi dell'86%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette un buon recall complessivo, considerando il peso delle diverse classi.
        -   F1-score: 0.80 indica una media ponderata degli F1-score per tutte le classi dell'80%. Questo valore tiene conto del supporto (numero di istanze) per ogni classe e riflette un bilanciamento complessivamente buono tra precisione e recall, considerando il peso delle diverse classi.

## Costruzione modello con selezione delle variabili esplicative via metodo RFE" (Recursive Feature Elimination with Cross-Validation)

RFE è un metodo di selezione delle variabili esplicative utilizzato nell'apprendimento automatico per identificare le caratteristice più rilevanti o informative per un determinato problema di predizione.

L'eliminazione ricorsiva delle feature (RFE) è un approccio iterativo che viene solitamente utilizzato in combinazione con un modello di apprendimento automatico.
Il Recursive Feature Elimination with Cross-Validation è una variante di RFE che utilizza la cross-validazione per valutare le performance del modello durante la selezione delle feature. L'algoritmo RFECV segue i seguenti passaggi:

1. Addestra un modello sul set completo di feature.
2. Valuta le performance del modello utilizzando la cross-validazione.
3. Calcola l'importanza delle feature.
4. Elimina le feature meno importanti.
5. Ripete i passaggi precedenti sul set ridotto di feature fino a raggiungere il numero desiderato di feature.
6. Seleziona il sottoinsieme di feature che ha ottenuto le migliori performance del modello sulla cross-validazione.

RFECV utilizza la cross-validazione per valutare le performance del modello su diversi sottoinsiemi di feature e seleziona automaticamente il numero ottimale di feature che massimizzano le prestazioni del modello.

RFECV è più robusto e adatto a selezionare il numero ideale di feature per il modello rispetto a RFE.
Questo può aiutare a migliorare le prestazioni del modello, ridurre la dimensionalità dei dati e mitigare il rischio di overfitting.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

# Creazione di un oggetto di regressione logistica
logreg = LogisticRegression()

# Definizione del selettore di feature RFE
selector = RFECV(logreg, step=1)

# Addestramento del selettore sul set di addestramento
selector.fit(X_train_d, y_train_d)

# Selezione delle variabili
selected_features = X_train_d.columns[selector.support_]

# Addestramento del modello di regressione logistica sulle variabili selezionate
model_rfecv = sm.GLM(y_train_d, X_train_d[selected_features], family=sm.families.Binomial())
fit_rfecv = model_rfecv.fit()

# Stampa il riassunto del modello
print(fit_rfecv.summary())

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1305.2

# Numero di parametri nel modello
num_parametri = 17+1

# Numero di osservazioni nel dataset di addestramento
n = len(y_train_d)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AICc

## Selezione delle variabili esplicative con il metodo Forward Selection

La Forward Selection è una tecnica di selezione delle feature utilizzata per costruire un modello incrementale, aggiungendo iterativamente le feature più rilevanti al modello:

1. Si inizia con un modello vuoto che non include alcuna feature.
2. Si addestra un modello di machine learning utilizzando ogni singola feature indipendentemente e si valuta le performance del modello (ad esempio, utilizzando una metrica come l'accuratezza, l'AUC, o l'errore quadratico medio).
3. Si seleziona la feature che offre la miglior performance del modello (ad esempio, quella con il valore più alto di AUC).
4. Si aggiunge questa feature al modello.
5. Si ripete il processo selezionando una nuova feature tra le feature rimanenti (quelle non ancora selezionate) e combinandola con le feature già selezionate nel modello.
6. Si continua l'iterazione, selezionando iterativamente la feature successiva che offre il miglior miglioramento delle performance del modello.
7. Si continua fino a quando si raggiunge un criterio di terminazione predefinito, ad esempio, un numero massimo di feature desiderato o quando non si osserva più un miglioramento significativo nelle performance del modello.

La Forward Selection è un processo incrementale che costruisce il modello passo dopo passo, aggiungendo le feature che migliorano maggiormente le performance del modello. È importante notare che la Forward Selection non tiene conto dell'effetto delle feature già selezionate sulle feature che verranno selezionate in seguito. Pertanto, potrebbe non selezionare l'insieme ottimale di feature.

È possibile utilizzare criteri di valutazione delle performance come l'AUC, l'accuratezza, l'errore quadratico medio o altre metriche appropriate per determinare quale feature aggiungere al modello in ogni passo.

È importante considerare che la Forward Selection può essere computazionalmente costosa, soprattutto se il numero di feature è elevato. Pertanto, possono essere utilizzati metodi di ottimizzazione e algoritmi efficienti per semplificare il processo di selezione delle feature.

In [ ]:
def forward_selection(X, y, initial_features=[]):
    remaining_features = set(X.columns)
    selected_features = list(initial_features)
    current_score = 0.0
    best_new_score = float('inf')  # Inizializzazione con un valore molto grande

    while remaining_features and current_score <= best_new_score:
        best_new_score = float('inf')  # Reimpostazione del valore all'inizio del ciclo
        best_feature = None

        for feature in remaining_features:
            model = sm.GLM(y, X[selected_features + [feature]], family=sm.families.Binomial())
            fit = model.fit()
            aic = fit.aic

            if aic < best_new_score:
                best_new_score = aic
                best_feature = feature

        if current_score <= best_new_score:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            current_score = best_new_score
        else:
            break

    model = sm.GLM(y, X[selected_features], family=sm.families.Binomial())
    final_fit = model.fit()
    return final_fit

In [ ]:
# Utilizzo della Forward_selection
selected_features = forward_selection(X_train, y_train)

# Stampa il riassunto del modello finale
print(selected_features.summary())

## Come calcolare l'AIC

In [ ]:
# Si calcola l'AICc (corrected Akaike Information Criterion) del modello completo per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1245.2

# Numero di parametri nel modello
num_parametri = 53+1

# Numero di osservazioni nel dataset di addestramento
n = len(y_train_d)

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri
AICc = AIC + (2 * num_parametri * (num_parametri + 1)) / (n - num_parametri - 1)

# Un modello con un valore di AICc più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AICc

In [ ]:
# Si calcola l'AIC (Akaike Information Criterion) per un futuro confronto con modelli alternativi.

# Log-likelihood del modello
log_likelihood = -1094.8

# Numero di parametri nel modello
num_parametri = 53+1

# Calcolo dell'AIC
AIC = -2 * log_likelihood + 2 * num_parametri

# Un modello con un valore di AIC più basso è generalmente preferito in quanto indica un migliore equilibrio
# tra bontà di adattamento e complessità del modello.

AIC

Per migliorare il modello a livello statistico, si possono considerare alcune azioni:

1. Esaminare la significatività delle variabili: Dai valori p, si possono identificare le variabili che potrebbero non essere significative per il tuo modello. Potrebbe essere utile rimuovere queste variabili o considerare trasformazioni delle stesse.

2. Esaminare la bontà di adattamento: Dai valori di devianza e Pearson chi2, si può valutare la bontà di adattamento del modello ai dati. Se questi valori sono molto elevati, potrebbe essere necessario considerare modelli alternativi o includere altre variabili esplicative per migliorare la precisione del modello.

3. Considerare l'aggiunta di altre variabili: Potresti esaminare se l'aggiunta di altre variabili esplicative potrebbe migliorare la capacità predittiva del tuo modello. Tuttavia, è importante valutare attentamente le variabili aggiuntive per garantire che siano pertinenti e non causino problemi come multicollinearità.

4. Valutare altre misure di bontà di adattamento: Oltre al Pseudo R-quadrato, si possono considerare altre misure di bontà di adattamento, come l'AIC (Akaike Information Criterion) o il BIC (Bayesian Information Criterion), per confrontare modelli alternativi e selezionare il migliore.

## Costruzione del modello completo attraverso la K-Fold Cross-Validation

La K-Fold cross-validation è una tecnica che suddivide il dataset in K parti uguali (fold). Il modello viene addestrato su K-1 fold e viene valutato sul fold rimanente. Questa operazione viene ripetuta K volte, utilizzando ogni volta un fold diverso come set di test e gli altri fold combinati come set di allenamento. Alla fine, vengono calcolate le prestazioni del modello su tutti i fold di test e ne viene fatta la media per ottenere una stima più affidabile delle prestazioni generali del modello.

K-Fold cross-validation è spesso utilizzata quando si dispone di un dataset relativamente piccolo o quando si desidera avere una stima più accurata delle prestazioni del modello rispetto a una singola divisione tra training e test set.

In [ ]:
hr_cv = hr_logit_dummy.copy()

In [ ]:
# Si procede con la normalizzazione delle caratteristiche
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

hr_cv[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']] = scaler.fit_transform(hr_cv[[ 'MonthlyIncome_Euro', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
           'DistanceFromHome','Age','Nr Ore']])

La normalizzazione viene eseguita nel seguente modo:

- Crea un'istanza della classe StandardScaler assegnandola alla variabile scaler.
- Applica la normalizzazione alle colonne specificate del dataframe hr_cv utilizzando il metodo fit_transform() della classe StandardScaler.
- Il risultato della normalizzazione viene assegnato di nuovo alle stesse colonne nel dataframe hr_cv.

La normalizzazione delle features rende i dati comparabili tra loro. Utilizzando lo StandardScaler, le colonne specificate vengono trasformate in modo tale che abbiano una media pari a 0 e una deviazione standard pari a 1. Questo aiuta a evitare che le variabili con valori più grandi abbiano un'influenza sproporzionata sui modelli.

In [ ]:
# Si aggiunge nel set di addestramento la costante per l'intercetta
hr_cv = sm.add_constant(hr_cv)

In [ ]:
# Si rinominano alcune variabili che hanno spazi o caratteri speciali nel nome
hr_cv.rename(columns={"Department_Research & Development": "Department_Research_and_Development"}, inplace=True)
hr_cv.rename(columns={"JobInvolvement_Very High": "JobInvolvement_VeryHigh"}, inplace=True)
hr_cv.rename(columns={"EnvironmentSatisfaction_Very High": "EnvironmentSatisfaction_VeryHigh"}, inplace=True)
hr_cv.rename(columns={"JobSatisfaction_Very High": "JobSatisfaction_VeryHigh"}, inplace=True)
hr_cv.rename(columns={"Education_Below College": "Education_BelowCollege"}, inplace=True)
hr_cv.rename(columns={"EducationField_Life Sciences": "EducationField_LifeSciences"}, inplace=True)
hr_cv.rename(columns={"EducationField_Technical Degree": "EducationField_TechnicalDegree"}, inplace=True)
hr_cv.rename(columns={"JobRole_Human Resources": "JobRole_HumanResources"}, inplace=True)
hr_cv.rename(columns={"JobRole_Laboratory Technician": "JobRole_LaboratoryTechnician"}, inplace=True)
hr_cv.rename(columns={"JobRole_Manufacturing Director": "JobRole_ManufacturingDirector"}, inplace=True)
hr_cv.rename(columns={"JobRole_Research Director": "JobRole_ResearchDirector"}, inplace=True)
hr_cv.rename(columns={"JobRole_Research Scientist": "JobRole_ResearchScientist"}, inplace=True)
hr_cv.rename(columns={"JobRole_Sales Executive": "JobRole_SalesExecutive"}, inplace=True)
hr_cv.rename(columns={"JobRole_Sales Representative": "JobRole_SalesRepresentative"}, inplace=True)
hr_cv.rename(columns={"Nr Ore": "Nr_Ore"}, inplace=True)

Per una scelta ottimale del valore K per la cross-validation si traccia la curva di validazione, eseguendo la K-Fold cross-validation per diversi valori di K

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# si definisce una lista di valori di K per cui si desidera eseguire la K-Fold cross-validation
K_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]  # Si può modificare questa lista con i valori di K che si desidera testare

# Si esegue la K-Fold cross-validation per ciascun valore di K e si calcola l'accuratezza media su tutti i fold.
mean_cv_scores = []

mean_cv_scores = []
std_cv_scores = []

for K in K_values:
    kf = KFold(n_splits=K, shuffle=True, random_state=42)
    cv_scores = []
    for train_index, test_index in kf.split(hr_cv):
        train_data = hr_cv.iloc[train_index]
        test_data = hr_cv.iloc[test_index]

        # Si addestra il modello sui dati di training.
        log_model = sm.GLM.from_formula(formula, data=train_data, family=sm.families.Binomial())
        fit_model = log_model.fit()

        # Si effettuano le predizioni sul fold di test.
        y_pred = (fit_model.predict(test_data) > 0.5).astype(int)

        # Si calcola l'accuratezza sul fold di test e la si salva nella lista dei punteggi.
        acc = accuracy_score(y_true=test_data['Attrition'], y_pred=y_pred)
        cv_scores.append(acc)

    # Si calcola l'accuratezza media e la deviazione standard e si aggiungono alle rispettive liste.
    mean_acc = np.mean(cv_scores)
    mean_cv_scores.append(mean_acc)
    std_acc = np.std(cv_scores)
    std_cv_scores.append(std_acc)

In [ ]:
# Si traccia la curva di validazione solo se ci sono valori in std_cv_scores
if len(std_cv_scores) > 0:
    plt.errorbar(K_values, mean_cv_scores, yerr=std_cv_scores, marker='o', linestyle='-', color='b')
    plt.xlabel('K (Numero di Fold)')
    plt.ylabel('Average Accuracy')
    plt.title('Curva di validazione')
    plt.show()
else:
    print("Impossibile tracciare la curva di validazione: assicurati di avere abbastanza campioni per la K-Fold cross-validation.")

L'accuratezza media è rappresentata dal punto blu, mentre le barre di errore indicano la deviazione standard dell'accuratezza. La Curva di validazione aiuta a identificare il valore di K che produce il miglior compromesso tra bias e varianza, e quindi il valore di K ottimale per la K-Fold cross-validation.

Il fatto che l'accuratezza media non aumenti significativamente all'aumentare del numero di fold (valori di K) suggerisce che il modello non generalizza bene.
La deviazione standard offre un'idea della variabilità delle prestazioni del modello su diversi fold. Un valore più alto di deviazione standard indica una maggiore variabilità e incertezza nelle prestazioni del modello su diversi fold. Al contrario, una deviazione standard più bassa suggerisce maggiore coerenza nelle prestazioni del modello.

In generale, si cerca di ottenere un equilibrio tra accuratezza media elevata e deviazione standard bassa. Se si ottiene un'alta accuratezza media e una bassa deviazione standard per un valore specifico di K, potrebbe essere una buona scelta per la K-Fold cross-validation del modello.

Nel caso in esame un buon compromesso sembra essere K = 3

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import numpy as np

# Definizione della formula
formula = 'Attrition ~ const + Nr_Ore + Age + DistanceFromHome + MonthlyIncome_Euro + NumCompaniesWorked + PercentSalaryHike + StockOptionLevel + TotalWorkingYears + TrainingTimesLastYear + YearsAtCompany + YearsSinceLastPromotion + YearsWithCurrManager + JobInvolvement_Low + JobInvolvement_Medium + JobInvolvement_VeryHigh + PerformanceRating_Outstanding + EnvironmentSatisfaction_Low + EnvironmentSatisfaction_Medium + EnvironmentSatisfaction_VeryHigh + JobSatisfaction_Low + JobSatisfaction_Medium + JobSatisfaction_VeryHigh + WorkLifeBalance_Best + WorkLifeBalance_Better + WorkLifeBalance_Good + BusinessTravel_Travel_Frequently + BusinessTravel_Travel_Rarely + Department_Research_and_Development + Department_Sales + Education_BelowCollege + Education_College + Education_Doctor + Education_Master + EducationField_LifeSciences + EducationField_Marketing + EducationField_Medical + EducationField_Other + EducationField_TechnicalDegree + Gender_Male + JobLevel_2 + JobLevel_3 + JobLevel_4 + JobLevel_5 + JobRole_HumanResources + JobRole_LaboratoryTechnician + JobRole_Manager + JobRole_ManufacturingDirector + JobRole_ResearchDirector + JobRole_ResearchScientist + JobRole_SalesExecutive + JobRole_SalesRepresentative + MaritalStatus_Married + MaritalStatus_Single'

# Si definisce la strategia di cross-validation (K-Fold con K=3)
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Si inizializza una lista per memorizzare le accuratezze della cross-validation
cv_accuracy_scores = []

# Si effettua la cross-validation manualmente
for train_index, test_index in kf.split(hr_cv):
    train_data = hr_cv.iloc[train_index]
    test_data = hr_cv.iloc[test_index]

    # Si crea il modello di regressione logistica con la formula corretta.
    CV_log_model = sm.GLM.from_formula(formula, data=train_data, family=sm.families.Binomial())

    # Si addestra il modello sui dati di training.
    fit_model_cv = CV_log_model.fit()

    # Si effettuano le predizioni sul fold di test.
    y_pred = (fit_model_cv.predict(test_data) > 0.5).astype(int)

    # Si calcola l'accuratezza sul fold di test e la si aggiunge alla lista.
    acc = accuracy_score(y_true=test_data['Attrition'], y_pred=y_pred)
    cv_accuracy_scores.append(acc)

# Si calcola l'accuratezza media e la deviazione standard dell'accuratezza
mean_cv_accuracy = np.mean(cv_accuracy_scores)
std_cv_accuracy = np.std(cv_accuracy_scores)

# Si visualizzano i risultati della cross-validation
print("Cross-Validation Scores:", cv_accuracy_scores)
print("Mean CV Accuracy:", mean_cv_accuracy)
print("Standard Deviation of CV Accuracy:", std_cv_accuracy)

I risultati della cross-validation che si sono ottenuti possono essere interpretati come segue:

* Cross-Validation Scores: è una lista dei punteggi di accuratezza ottenuti su ciascun fold durante la cross-validation. Si è effettuato una cross-validation a 3 fold, quindi ci sono 3 punteggi di accuratezza. I valori [0.8523809523809524, 0.8421768707482993, 0.8461538461538461] corrispondono alle accuratezze ottenute per i tre fold.

* Mean CV Accuracy: è il punteggio medio di accuratezza ottenuto su tutti i fold durante la cross-validation. In questo caso, l'accuratezza media è di circa 0.8469, che indica che in media il modello ha previsto correttamente l'attrition di circa l'84.69% dei dipendenti nei dati di test durante la cross-validation.

* Standard Deviation of CV Accuracy: Questo è il valore della deviazione standard dell'accuratezza ottenuto su tutti i fold durante la cross-validation. La deviazione standard misura la variabilità dei punteggi di accuratezza tra i fold. In questo caso, la deviazione standard è di circa 0.0042, che indica che i punteggi di accuratezza nei fold hanno una bassa variabilità tra loro, il che è un buon segno di stabilità delle previsioni del modello.

In sintesi, si è ottenuto un'accuratezza media di circa l'84.69% durante la cross-validation, e i punteggi di accuratezza nei fold sono relativamente vicini tra loro, suggerendo che il modello ha una buona stabilità e coerenza nelle previsioni. Tuttavia, l'accuratezza può variare leggermente tra i fold, come indicato dalla deviazione standard.